# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [14221]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [14222]:
!pg_ctl -D $DB_DIR status

pg_ctl: no server running


## Stop any existing PostgreSQL process

In [14223]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

pg_ctl: PID file "mylocal_db/postmaster.pid" does not exist
Is server running?
Nothing to stop.


## Initialize the database if necessary

In [14224]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [14225]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [14226]:
import subprocess
import time
import atexit

def stop_postgres():
    !pg_ctl -D $DB_DIR stop

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen(["pg_ctl", "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)
time.sleep(5)

## Create the "notebook" user with full permissions

In [14227]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

ERROR:  role "notebook" already exists
User already exists.
ALTER ROLE


# Connect

## Load & Configure SQLMagic

In [14228]:
%load_ext sql
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

## Create Fresh sahuagin Database

In [14229]:
!psql -U notebook -d postgres -c "SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'sahuagin';"
!psql -U notebook -d postgres -c "DROP DATABASE IF EXISTS sahuagin;"
!psql -U notebook -d postgres -c "CREATE DATABASE sahuagin;"

 pg_terminate_backend 
----------------------
 t
(1 row)

DROP DATABASE
CREATE DATABASE


## Connect to sahuagin Database

In [14230]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

## Attribute Type

In [14231]:
%%sql

DROP TYPE IF EXISTS attr_type CASCADE;
CREATE TYPE attr_type AS ENUM ('discrete','continuous');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

/Users/nashspence/miniforge3/envs/jupyter-env/lib/python3.11/site-packages/sql/connection/connection.py:899: JupySQLRollbackPerformed: Server closed connection. JupySQL executed a ROLLBACK operation.
  warnings.warn(


++
||
++
++

## Attribute

In [14232]:
%%sql

DROP TABLE IF EXISTS attribute CASCADE;
CREATE TABLE attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE,
    type attr_type NOT NULL,
    decimals integer,
    max_value double precision,
    min_value double precision,
    mode_value double precision,
    concentration double precision,
    skew double precision,
    units varchar(255),
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND decimals IS NULL
            AND max_value IS NULL
            AND min_value IS NULL
            AND mode_value IS NULL
            AND concentration IS NULL
            AND skew IS NULL
            AND units IS NULL
        )
        OR (
            type = 'continuous'
            AND decimals IS NOT NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
            AND mode_value IS NOT NULL
            AND concentration IS NOT NULL
            AND skew IS NOT NULL
        )
    )
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Span

In [14233]:
%%sql

DROP TABLE IF EXISTS span CASCADE;
CREATE TABLE span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    label varchar(255) NOT NULL,
    type attr_type NOT NULL,
    is_pinned boolean,
    weight double precision,
    max_value double precision,
    min_value double precision,
    CONSTRAINT uq_span_label_within_attribute UNIQUE (attribute_id, label),
    CONSTRAINT fk_span_to_attribute FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND is_pinned IS NOT NULL
            AND weight IS NOT NULL
            AND max_value IS NULL
            AND min_value IS NULL
        )
        OR (
            type = 'continuous'
            AND is_pinned IS NULL
            AND weight IS NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
        )
    )
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant

In [14234]:
%%sql

DROP TABLE IF EXISTS variant CASCADE;
CREATE TABLE variant (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant Attribute

In [14235]:
%%sql

DROP TABLE IF EXISTS variant_attr CASCADE;
CREATE TABLE variant_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    name varchar(255) NOT NULL,
    causation_index integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_name UNIQUE (variant_id, name),
    CONSTRAINT unique_variant_index UNIQUE (variant_id, causation_index),
    CONSTRAINT fk_variant_attr_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_variant_attr_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Subvariant Span

In [14236]:
%%sql

DROP TABLE IF EXISTS subvariant_span CASCADE;
CREATE TABLE subvariant_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_span UNIQUE (variant_id, span_id),
    CONSTRAINT fk_varattr_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Possible Attribute

In [14237]:
%%sql

DROP TABLE IF EXISTS possible_attr CASCADE;
CREATE TABLE possible_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_attr_id integer NOT NULL,
    subvariant_span_id integer,
    parent_possible_attr_id integer,
    CONSTRAINT unique_possible_attr UNIQUE (parent_possible_attr_id, subvariant_span_id, variant_attr_id),
    CONSTRAINT fk_possible_attr_va FOREIGN KEY (variant_attr_id) REFERENCES variant_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_possible_attr_ss FOREIGN KEY (subvariant_span_id) REFERENCES subvariant_span (id) ON DELETE CASCADE,
    CONSTRAINT fk_possible_attr_parent FOREIGN KEY (parent_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Effect

In [14238]:
%%sql

DROP TABLE IF EXISTS effect CASCADE;
CREATE TABLE effect (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE,
    variant_id integer NOT NULL,
    span_id integer NOT NULL,
    to_modify_possible_attr_id integer NOT NULL,
    activating_possible_attr_id integer NOT NULL,
    CONSTRAINT uq_effect UNIQUE (span_id, to_modify_possible_attr_id, activating_possible_attr_id),
    CONSTRAINT fk_effect_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_psattr_to_modify FOREIGN KEY (to_modify_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_psattr_activating FOREIGN KEY (activating_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delta Parameter Effect

In [14239]:
%%sql

DROP TABLE IF EXISTS delta_param_eft CASCADE;
CREATE TABLE delta_param_eft (    
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    delta_mode double precision NOT NULL,
    delta_conc double precision NOT NULL,
    delta_skew double precision NOT NULL,
    CONSTRAINT fk_var_continuous_attr_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Span Effect

In [14240]:
%%sql

DROP TABLE IF EXISTS add_span_eft CASCADE;
CREATE TABLE add_span_eft (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_activated_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_activated_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delta Weight Effect

In [14241]:
%%sql

DROP TABLE IF EXISTS delta_weight_eft CASCADE;
CREATE TABLE delta_weight_eft (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    delta_weight double precision NOT NULL,
    CONSTRAINT fk_var_delta_weight_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_delta_weight_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Remove Span Effect

In [14242]:
%%sql

DROP TABLE IF EXISTS remove_span_eft CASCADE;
CREATE TABLE remove_span_eft (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_inactive_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_inactive_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity

In [14243]:
%%sql

DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_id integer NOT NULL,
    name varchar(255) NOT NULL UNIQUE,
    CONSTRAINT fk_entity_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity Group

In [14244]:
%%sql

DROP TABLE IF EXISTS entity_group CASCADE;
CREATE TABLE entity_group (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity Group Link

In [14245]:
%%sql

DROP TABLE IF EXISTS entity_group_link CASCADE;
CREATE TABLE entity_group_link (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    entity_group_id integer NOT NULL,
    CONSTRAINT unique_link_entity_group UNIQUE (entity_group_id, entity_id),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE,
    CONSTRAINT fk_entity_group FOREIGN KEY (entity_group_id) REFERENCES entity_group (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity State

In [14246]:
%%sql

DROP TABLE IF EXISTS entity_state CASCADE;
CREATE TABLE entity_state (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    time double precision NOT NULL,
    CONSTRAINT unique_entity_time UNIQUE (entity_id, time),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attribute Value

In [14247]:
%%sql

DROP TABLE IF EXISTS attr_value CASCADE;
CREATE TABLE attr_value (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_state_id integer NOT NULL,
    numeric_value double precision,
    span_id integer,
    possible_attr_id integer NOT NULL,
    CONSTRAINT unique_state_possible_attr UNIQUE (entity_state_id, possible_attr_id),
    CONSTRAINT fk_attr_val_entity_state FOREIGN KEY (entity_state_id) REFERENCES entity_state (id) ON DELETE CASCADE,
    CONSTRAINT fk_attr_val_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_attr_val_possible_attr FOREIGN KEY (possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attribute Value Lock

In [14248]:
%%sql

DROP TABLE IF EXISTS attr_val_lock CASCADE;
CREATE TABLE attr_val_lock (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    locked_attr_val_id integer NOT NULL,
    locking_attr_val_id integer,
    CONSTRAINT unique_attr_val_lock UNIQUE (locking_attr_val_id, locked_attr_val_id),
    CONSTRAINT fk_attr_val_lock_locked_attr_val FOREIGN KEY (locked_attr_val_id) REFERENCES attr_value (id) ON DELETE CASCADE,
    CONSTRAINT fk_attr_val_lock_locking_attr_val FOREIGN KEY (locking_attr_val_id) REFERENCES attr_value (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Triggers

## Delete Orphan possible_attr

In [14249]:
%%sql

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_effect()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    -- Check the to_modify_possible_attr_id column
    IF OLD.to_modify_possible_attr_id IS NOT NULL THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.to_modify_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.to_modify_possible_attr_id;
        END IF;
    END IF;
    
    -- Check the activating_possible_attr_id column (if different)
    IF OLD.activating_possible_attr_id IS NOT NULL
       AND (OLD.activating_possible_attr_id <> OLD.to_modify_possible_attr_id) THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.activating_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.activating_possible_attr_id;
        END IF;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_attr_val()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    SELECT
        (SELECT COUNT(*) FROM effect 
           WHERE to_modify_possible_attr_id = OLD.possible_attr_id
              OR activating_possible_attr_id = OLD.possible_attr_id)
      + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.possible_attr_id)
      INTO orphan_count;
      
    IF orphan_count = 0 THEN
        DELETE FROM possible_attr WHERE id = OLD.possible_attr_id;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_effect
AFTER DELETE ON effect
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_effect();

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_attr_val
AFTER DELETE ON attr_value
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_attr_val();

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delete Orphan Span

In [14250]:
%%sql

CREATE OR REPLACE FUNCTION delete_orphaned_span() RETURNS TRIGGER AS $$
BEGIN
    DELETE FROM span WHERE id = OLD.span_id;
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_delete_span_after_activation_delete
AFTER DELETE ON add_span_eft
FOR EACH ROW
EXECUTE FUNCTION delete_orphaned_span();

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Create Indexes

## idx_attribute_name

In [14251]:
%%sql

DROP INDEX IF EXISTS idx_attribute_name;
CREATE INDEX idx_attribute_name ON attribute (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attribute_id

In [14252]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_id;
CREATE INDEX idx_span_attribute_id ON span (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type

In [14253]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type;
CREATE INDEX idx_span_attr_type ON span(attribute_id, type, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type

In [14254]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type;
CREATE INDEX idx_span_attr_type ON span(attribute_id, type, min_value, max_value);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned

In [14255]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned;
CREATE INDEX idx_span_attr_type_pinned ON span(attribute_id, type, is_pinned);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned_wl

In [14256]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned_wl;
CREATE INDEX idx_span_attr_type_pinned_wl ON span(attribute_id, type, is_pinned, weight, label);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_name

In [14257]:
%%sql

DROP INDEX IF EXISTS idx_variant_name;
CREATE INDEX idx_variant_name ON variant (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_attribute

In [14258]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_attribute;
CREATE INDEX idx_variant_attr_attribute ON variant_attr (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_variant

In [14259]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_variant;
CREATE INDEX idx_variant_attr_variant ON variant_attr (variant_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_variant_causation

In [14260]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_variant_causation;
CREATE INDEX idx_variant_attr_variant_causation ON variant_attr (variant_id, causation_index);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_subvariant_span_span_id

In [14261]:
%%sql

DROP INDEX IF EXISTS idx_subvariant_span_span_id;
CREATE INDEX idx_subvariant_span_span_id ON subvariant_span (span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_variant_attr

In [14262]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_variant_attr;
CREATE INDEX idx_possible_attr_variant_attr ON possible_attr (variant_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_vaid

In [14263]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_vaid;
CREATE INDEX idx_possible_attr_vaid ON possible_attr (variant_attr_id, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_parent

In [14264]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_parent;
CREATE INDEX idx_possible_attr_parent ON possible_attr(parent_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_uq_possible_attr

In [14265]:
%%sql

DROP INDEX IF EXISTS idx_uq_possible_attr;
CREATE INDEX idx_uq_possible_attr ON possible_attr (
    variant_attr_id,
    subvariant_span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_va_pss

In [14266]:
%%sql

DROP INDEX IF EXISTS idx_uq_possible_attr;
CREATE INDEX idx_possible_attr_va_pss
  ON possible_attr(variant_attr_id, parent_possible_attr_id, subvariant_span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_var_cont_variation

In [14267]:
%%sql

DROP INDEX IF EXISTS idx_var_cont_variation;
CREATE INDEX idx_var_cont_variation ON delta_param_eft (effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_effect_activating_psattr

In [14268]:
%%sql

DROP INDEX IF EXISTS idx_effect_activating_psattr;
CREATE INDEX idx_effect_activating_psattr ON effect(activating_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_effect_to_modify_psattr

In [14269]:
%%sql

DROP INDEX IF EXISTS idx_effect_to_modify_psattr;
CREATE INDEX idx_effect_to_modify_psattr ON effect(to_modify_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_add_span_eft_effect

In [14270]:
%%sql

DROP INDEX IF EXISTS idx_add_span_eft_effect;
CREATE INDEX idx_add_span_eft_effect
  ON add_span_eft(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_add_span_eft_varid_spanid

In [14271]:
%%sql

DROP INDEX IF EXISTS idx_add_span_eft_effid_spanid;
CREATE INDEX idx_add_span_eft_effid_spanid ON add_span_eft (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_remove_span_eft_effect

In [14272]:
%%sql

DROP INDEX IF EXISTS idx_remove_span_eft_effect;
CREATE INDEX idx_remove_span_eft_effect
  ON remove_span_eft(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_inactive

In [14273]:
%%sql

DROP INDEX IF EXISTS idx_variation_inactive;
CREATE INDEX idx_variation_inactive ON remove_span_eft (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

/Users/nashspence/miniforge3/envs/jupyter-env/lib/python3.11/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


++
||
++
++

## idx_delta_weight_eft_effect

In [14274]:
%%sql

DROP INDEX IF EXISTS idx_delta_weight_eft_effect;
CREATE INDEX idx_delta_weight_eft_effect
  ON delta_weight_eft(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_delta_weight_eft_varid_spanid

In [14275]:
%%sql

DROP INDEX IF EXISTS idx_delta_weight_eft_effid_spanid;
CREATE INDEX idx_delta_weight_eft_effid_spanid ON delta_weight_eft (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta

In [14276]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta;
CREATE INDEX idx_variation_delta ON delta_weight_eft (
    effect_id,
    span_id,
    delta_weight
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_id

In [14277]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_id;
CREATE INDEX idx_entity_state_entity_id ON entity_state (entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_time

In [14278]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_time;
CREATE INDEX idx_entity_state_entity_time ON entity_state (entity_id, time);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_value_entity_state_id

In [14279]:
%%sql

DROP INDEX IF EXISTS idx_attr_value_entity_state_id;
CREATE INDEX idx_attr_value_entity_state_id ON attr_value (entity_state_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_val_state_vaid

In [14280]:
%%sql

DROP INDEX IF EXISTS idx_attr_val_state_vaid;
CREATE INDEX idx_attr_val_state_vaid ON attr_value (
    entity_state_id,
    possible_attr_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_val_state_attr_span

In [14281]:
%%sql

DROP INDEX IF EXISTS idx_attr_val_state_attr_span;
CREATE INDEX idx_attr_val_state_attr_span ON attr_value (
    entity_state_id,
    possible_attr_id,
    span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_val_lock_locked

In [14282]:
%%sql

DROP INDEX IF EXISTS idx_attr_val_lock_locked;
CREATE INDEX idx_attr_val_lock_locked ON attr_val_lock (locked_attr_val_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_val_lock_locking

In [14283]:
%%sql

DROP INDEX IF EXISTS idx_attr_val_lock_locking;
CREATE INDEX idx_attr_val_lock_locking ON attr_val_lock (locking_attr_val_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_attr_val_lock_locked_locking

In [14284]:
%%sql

DROP INDEX IF EXISTS idx_attr_val_lock_locked_locking;
CREATE INDEX idx_attr_val_lock_locked_locking ON attr_val_lock (
    locked_attr_val_id,
    locking_attr_val_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_group_link_group_id_entity_id

In [14285]:
%%sql

DROP INDEX IF EXISTS idx_entity_group_link_group_id_entity_id;
CREATE INDEX idx_entity_group_link_group_id_entity_id
  ON entity_group_link(entity_group_id, entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Debug

## Create Log Table

In [14286]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Create Log Function

In [14287]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Clear Log Table

In [14288]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14289]:
%%sql

SELECT * FROM debug_log;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

id,log_time,procedure_name,log_message


# Create Functions & Procedures

## Discrete Attribute

### Add

In [14290]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_attribute(
    in_name       VARCHAR(255),
    in_spans      JSON  -- JSON array: either [["span1", 10], ["span2", 20, "subvariantX"]] or ["span1", "span2"]
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_attr_id       INTEGER;
    span_count        INTEGER;
    span_label        VARCHAR(255);
    span_weight       DOUBLE PRECISION;
    normalized_weight DOUBLE PRECISION;
    total_weight      DOUBLE PRECISION := 0;
    first_elem_type   TEXT;
    spans_json        JSON;
    i                 INTEGER;
BEGIN
    -- Create the new attribute.
    INSERT INTO attribute (name, type)
      VALUES (in_name, 'discrete')
      RETURNING id INTO new_attr_id;

    IF in_spans IS NOT NULL THEN
        spans_json := in_spans::json;
        span_count := json_array_length(spans_json);

        IF span_count > 0 THEN
            first_elem_type := json_typeof(spans_json->0);
        ELSE
            first_elem_type := '';
        END IF;

        IF first_elem_type = 'array' THEN
            -- First pass: compute the total weight.
            FOR i IN 0..span_count - 1 LOOP
                total_weight := total_weight + (((spans_json->i)->>1)::DOUBLE PRECISION);
            END LOOP;

            -- Second pass: insert spans with normalized weight.
            FOR i IN 0..span_count - 1 LOOP
                span_label := (spans_json->i)->>0;
                span_weight := ((spans_json->i)->>1)::DOUBLE PRECISION;
                normalized_weight := span_weight / total_weight;
                INSERT INTO span (attribute_id, label, type, is_pinned, weight)
                  VALUES (new_attr_id, span_label, 'discrete', false, normalized_weight);
                
                -- If a third element exists, call set_subvariant_span.
                IF json_array_length(spans_json->i) > 2 THEN
                    CALL set_span_as_subvariant(in_name, span_label, (spans_json->i)->>2);
                END IF;
            END LOOP;
        ELSE
            -- For a JSON array of labels only, call the helper procedure for each span.
            FOR i IN 0..span_count - 1 LOOP
                span_label := spans_json->>i;
                CALL add_discrete_span(new_attr_id, span_label);
            END LOOP;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [14291]:
%%sql

CREATE OR REPLACE FUNCTION get_discrete_attribute(p_attribute_name varchar)
RETURNS TABLE (
    attribute   varchar,
    type        attr_type,
    span_label  varchar,
    span_weight double precision,
    is_pinned      boolean
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      s.label,
      s.weight,
      s.is_pinned
    FROM attribute a
    JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND NOT EXISTS (
          SELECT 1 
          FROM add_span_eft ase 
          WHERE ase.span_id = s.id
      );
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [14292]:
%%sql

CREATE OR REPLACE PROCEDURE delete_discrete_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'discrete';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Redistribute Unpinned Spans

In [14293]:
%%sql

CREATE OR REPLACE FUNCTION redistribute_unpinned_spans(
    p_attribute_id INTEGER,
    p_modified_span_id INTEGER,
    p_new_weight DOUBLE PRECISION,
    p_old_weight DOUBLE PRECISION
) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
    v_pinned_weight DOUBLE PRECISION;
    v_target         DOUBLE PRECISION;
BEGIN
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    -- The available weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    
    -- If there’s only one unpinned span (v_target equals p_old_weight), then
    -- the new weight must equal the entire target.
    IF (v_target - p_old_weight) = 0 THEN
       IF p_new_weight <> v_target THEN
           RAISE EXCEPTION 'Cannot change weight; only one unpinned span exists and its weight must be %', v_target;
       END IF;
    END IF;
    
    -- Update all unpinned discrete spans:
    -- • The modified span gets the new weight.
    -- • All others are scaled proportionally.
    UPDATE span
       SET weight = CASE 
                      WHEN id = p_modified_span_id THEN p_new_weight
                      ELSE weight * ((v_target - p_new_weight) / (v_target - p_old_weight))
                    END
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Discrete Span to Attribute

In [14294]:
%%sql

CREATE OR REPLACE PROCEDURE add_disc_span_to_attr(
    p_attribute_id INTEGER,
    p_label VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_span_id     INTEGER;
    v_total_weight  DOUBLE PRECISION;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_count_old     INTEGER;
    candidate       DOUBLE PRECISION;
BEGIN
    -- Compute the total weight for all discrete spans for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_total_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete';

    -- If no spans exist, simply insert the first span with weight 1.
    IF v_total_weight = 0 THEN
        INSERT INTO span(attribute_id, label, type, is_pinned, weight)
        VALUES (p_attribute_id, p_label, 'discrete', false, 1.0);
        RETURN;
    END IF;
    
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
    
    -- Calculate the available target weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    IF v_target <= 0 THEN
        RAISE EXCEPTION 'No available weight for unpinned spans (v_target = %)', v_target;
    END IF;
    
    -- Count the existing non-pinned discrete spans.
    SELECT COUNT(*)
      INTO v_count_old
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
    
    -- Insert the new unpinned span with a temporary weight of 0.
    INSERT INTO span(attribute_id, label, type, is_pinned, weight)
    VALUES (p_attribute_id, p_label, 'discrete', false, 0.0)
    RETURNING id INTO new_span_id;
    
    -- Compute the candidate weight for the new span.
    candidate := v_target / (v_count_old + 1);
    
    -- Redistribute the weights among all unpinned spans
    PERFORM redistribute_unpinned_spans(p_attribute_id, new_span_id, candidate, 0);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Modify Discrete Span Weight

In [14295]:
%%sql

CREATE OR REPLACE PROCEDURE modify_disc_span_weight(
    p_span_id    INTEGER,
    p_new_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_current_weight DOUBLE PRECISION;
    v_is_pinned    BOOLEAN;
    v_type         TEXT;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_other_sum     DOUBLE PRECISION;
BEGIN
    -- Get the current span's details.
    SELECT attribute_id, weight, is_pinned, type
      INTO v_attribute_id, v_current_weight, v_is_pinned, v_type
      FROM span
     WHERE id = p_span_id;
     
    -- Ensure the span exists, is of type 'discrete' and is not pinned.
    IF NOT FOUND OR v_type <> 'discrete' OR v_is_pinned THEN
       RAISE EXCEPTION 'Span % not found or not a modifiable discrete span', p_span_id;
    END IF;
    
    -- Compute the total pinned weight for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    v_target := 1.0 - v_pinned_weight;
    
    -- Validate the new weight is between 0 and the available target.
    IF p_new_weight < 0 OR p_new_weight > v_target THEN
       RAISE EXCEPTION 'Invalid new weight: % (must be between 0 and %)', p_new_weight, v_target;
    END IF;
    
    -- Compute the total weight for all other unpinned spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_other_sum
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = false
       AND id <> p_span_id;
       
    IF v_other_sum = 0 THEN
       -- There is only one unpinned span. Its weight must equal the available target.
       IF p_new_weight <> v_target THEN
         RAISE EXCEPTION 'Only one unpinned span exists. Its weight must be %', v_target;
       ELSE
         UPDATE span SET weight = p_new_weight WHERE id = p_span_id;
       END IF;
    ELSE
       -- Use the helper function to redistribute weights.
       PERFORM redistribute_unpinned_spans(v_attribute_id, p_span_id, p_new_weight, v_current_weight);
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Continuous Attribute

### Add

In [14296]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_attribute(
    p_name           VARCHAR(255),
    p_min_value      DOUBLE PRECISION,
    p_mode_value     DOUBLE PRECISION,
    p_max_value      DOUBLE PRECISION,
    p_concentration  DOUBLE PRECISION,
    p_skew           DOUBLE PRECISION,
    p_decimals       INTEGER,
    p_units          VARCHAR(255),
    p_spans          JSON DEFAULT NULL  -- JSON array shortcut for continuous spans; tuples: [label, max_boundary, (optional subvariant)]
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_id   INTEGER;
    span_count  INTEGER;
    i           INTEGER;
    current_min DOUBLE PRECISION;
    new_max     DOUBLE PRECISION;
    span_label  VARCHAR(255);
    spans_json  JSON;
BEGIN
    -- Insert the continuous attribute record.
    INSERT INTO attribute
      (name, type, decimals, max_value, min_value,
       mode_value, concentration, skew, units)
    VALUES
      (p_name, 'continuous', p_decimals, p_max_value,
       p_min_value, p_mode_value, p_concentration, p_skew, p_units)
    RETURNING id INTO v_attr_id;
    
    IF p_spans IS NOT NULL THEN
        spans_json := p_spans::json;
        span_count := json_array_length(spans_json);
        current_min := p_min_value;
        
        FOR i IN 0..span_count - 1 LOOP
            -- Each JSON tuple is assumed to be [label, new_max, (optional subvariant)].
            span_label := (spans_json->i)->>0;
            new_max := ((spans_json->i)->>1)::DOUBLE PRECISION;
            
            -- Validate that the new maximum is in the valid interval.
            IF new_max <= current_min OR new_max > p_max_value THEN
                RAISE EXCEPTION 'Invalid span range: new_max (%) is not in the valid interval (% - %)',
                    new_max, current_min, p_max_value;
            END IF;
            
            -- Insert the continuous span.
            INSERT INTO span(attribute_id, label, type, min_value, max_value)
            VALUES(v_attr_id, span_label, 'continuous', current_min, new_max);
            
            -- If a third element exists, call set_subvariant_span.
            IF json_array_length(spans_json->i) > 2 THEN
                CALL set_span_as_subvariant(p_name, span_label, (spans_json->i)->>2);
            END IF;
            
            current_min := new_max;
        END LOOP;
        
        -- If the JSON array does not reach the attribute's overall max, raise an exception.
        IF current_min < p_max_value THEN
            RAISE EXCEPTION 'Incomplete continuous span definitions: final span max is % which is less than the attribute maximum %', current_min, p_max_value;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [14297]:
%%sql

CREATE OR REPLACE FUNCTION get_continuous_attribute(p_attribute_name varchar)
RETURNS TABLE (
    name           varchar,
    type           attr_type,
    min            double precision,
    mode           double precision,
    max            double precision,
    concentration  double precision,
    skew           double precision,
    units          varchar,
    decimals       integer,
    span_label     varchar,
    span_min       double precision,
    span_max       double precision
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      a.min_value,
      a.mode_value,
      a.max_value,
      a.concentration,
      a.skew,
      a.units,
      a.decimals,
      s.label,
      s.min_value,
      s.max_value
    FROM attribute a
    LEFT JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND a.type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [14298]:
%%sql

CREATE OR REPLACE PROCEDURE delete_continuous_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Continuous Span to Attribute

In [14299]:
%%sql

CREATE OR REPLACE PROCEDURE add_cont_span_to_attr(
    p_attribute_id integer,
    p_label varchar(255),
    p_new_min double precision,
    p_new_max double precision
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_min double precision;
    v_attr_max double precision;
    v_count integer;
    v_first_id integer;
    v_first_min double precision;
    v_first_max double precision;
    v_last_id integer;
    v_last_min double precision;
    v_last_max double precision;
BEGIN
    -- Get attribute’s overall range (continuous only)
    SELECT min_value, max_value
      INTO v_attr_min, v_attr_max
      FROM attribute
     WHERE id = p_attribute_id
       AND type = 'continuous'
     LIMIT 1;
     
    IF v_attr_min IS NULL THEN
        RAISE EXCEPTION 'Attribute not found or not continuous';
    END IF;
    
    -- Validate requested span
    IF p_new_min < v_attr_min OR p_new_max > v_attr_max OR p_new_min >= p_new_max THEN
        RAISE EXCEPTION 'Invalid span range';
    END IF;
    
    -- If no continuous spans exist, insert one covering the full range
    SELECT COUNT(*) INTO v_count
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous';
       
    IF v_count = 0 THEN
        INSERT INTO span(attribute_id, label, type, min_value, max_value)
        VALUES(p_attribute_id, p_label, 'continuous', v_attr_min, v_attr_max);
        RETURN;
    END IF;
    
    -- Adjust overlapping spans: first overlapping span (ordered by min_value)
    SELECT id, min_value, max_value
      INTO v_first_id, v_first_min, v_first_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY min_value ASC
     LIMIT 1;
     
    IF v_first_min < p_new_min THEN
        UPDATE span SET max_value = p_new_min WHERE id = v_first_id;
    END IF;
    
    -- Adjust overlapping spans: last overlapping span (ordered by max_value)
    SELECT id, min_value, max_value
      INTO v_last_id, v_last_min, v_last_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY max_value DESC
     LIMIT 1;
     
    IF v_last_max > p_new_max THEN
        UPDATE span SET min_value = p_new_max WHERE id = v_last_id;
    END IF;
    
    -- Remove spans fully covered by the new span
    DELETE FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND min_value >= p_new_min
       AND max_value <= p_new_max;
       
    INSERT INTO span(attribute_id, label, type, min_value, max_value)
    VALUES(p_attribute_id, p_label, 'continuous', p_new_min, p_new_max);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Link Attribute to Variant

In [14300]:
%%sql

CREATE OR REPLACE PROCEDURE link_attr_to_variant(
    p_variant_id   INTEGER,
    p_attribute_id INTEGER,
    p_name         VARCHAR,
    p_position     INTEGER DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_count    INTEGER;
    v_position INTEGER;
BEGIN
    -- Check if a variant attribute with the same name is already linked.
    IF EXISTS (
        SELECT 1 FROM variant_attr
         WHERE variant_id = p_variant_id
           AND name = p_name
    ) THEN
        RAISE EXCEPTION 'Variant attribute with name "%" already exists for variant %', p_name, p_variant_id;
    END IF;

    SELECT COUNT(*) INTO v_count
      FROM variant_attr
     WHERE variant_id = p_variant_id;
    
    IF p_position IS NULL THEN
        v_position := v_count;  -- append at end
    ELSE
        v_position := p_position;
        IF v_position < 0 THEN 
            v_position := 0;
        END IF;
        IF v_position > v_count THEN
            v_position := v_count;
        END IF;
        UPDATE variant_attr
           SET causation_index = causation_index + 1
         WHERE variant_id = p_variant_id
           AND causation_index >= v_position;
    END IF;

    INSERT INTO variant_attr(attribute_id, name, causation_index, variant_id)
    VALUES (p_attribute_id, p_name, v_position, p_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_effects_from_resolved_path

In [14301]:
%%sql

CREATE OR REPLACE FUNCTION get_effects_from_resolved_path(
    p_path int[][]
)
RETURNS TABLE(
    idx int,
    id int,
    name varchar,
    span_id int,
    relationship text
)
AS $$
DECLARE
    n int;
    i int;
    j int;
    overall_variant int;
    root_pa int;
    current_pa int;
BEGIN
    n := array_length(p_path, 1);
    IF n IS NULL OR n < 1 THEN
        RAISE EXCEPTION 'p_path must contain at least one element';
    END IF;

    -- Get overall variant_id from the canonical root (first row)
    SELECT va.variant_id
      INTO overall_variant
      FROM variant_attr va
     WHERE va.id = p_path[1][2];

    -- Iterate from the end (leaf) toward the beginning (root)
    FOR i IN REVERSE n..1 LOOP
        -- Find the corresponding canonical possible_attr for this ancestor level
        SELECT pa.id
          INTO root_pa
          FROM possible_attr pa
          JOIN variant_attr va ON pa.variant_attr_id = va.id
         WHERE pa.parent_possible_attr_id IS NULL
           AND pa.variant_attr_id = p_path[i][2]
           AND va.variant_id = overall_variant
         LIMIT 1;
        IF root_pa IS NULL THEN
            CONTINUE;
        END IF;

        current_pa := root_pa;
        -- Traverse forward from the current ancestor to the leaf using the original path
        FOR j IN i+1..n LOOP
            SELECT pa.id
              INTO current_pa
              FROM possible_attr pa
             WHERE pa.parent_possible_attr_id = current_pa
               AND pa.variant_attr_id = p_path[j][2]
               AND COALESCE(pa.subvariant_span_id, 0) = p_path[j][1]
             LIMIT 1;
            IF current_pa IS NULL THEN
                EXIT;
            END IF;
        END LOOP;

        IF current_pa IS NOT NULL THEN
            RETURN QUERY
            SELECT (n - i + 1) AS idx,
                   e.id,
                   e.name,
                   e.span_id,
                   'activates' AS relationship
              FROM effect e
             WHERE e.activating_possible_attr_id = current_pa
             ORDER BY e.id;

            RETURN QUERY
            SELECT (n - i + 1) AS idx,
                   e.id,
                   e.name,
                   e.span_id,
                   'effected_by' AS relationship
              FROM effect e
             WHERE e.to_modify_possible_attr_id = current_pa
             ORDER BY e.id;
        END IF;
    END LOOP;
    RETURN;
END;
$$ LANGUAGE plpgsql;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Same Slot

In [14302]:
%%sql

CREATE OR REPLACE FUNCTION same_slot(
    p_candidate_id INTEGER,
    p_reference_id INTEGER
)
RETURNS BOOLEAN
LANGUAGE plpgsql
AS $$
DECLARE
    candidate_slot  INTEGER;
    reference_slot  INTEGER;
BEGIN
    -- If the candidate is a fallback (no possible_attr row), use abs(id)
    IF p_candidate_id < 0 THEN
       candidate_slot := -p_candidate_id;
    ELSE
       SELECT COALESCE(parent_possible_attr_id, id)
         INTO candidate_slot
         FROM possible_attr
        WHERE id = p_candidate_id;
    END IF;

    -- Same for the reference.
    IF p_reference_id < 0 THEN
       reference_slot := -p_reference_id;
    ELSE
       SELECT COALESCE(parent_possible_attr_id, id)
         INTO reference_slot
         FROM possible_attr
        WHERE id = p_reference_id;
    END IF;

    RETURN candidate_slot = reference_slot;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant

### Add

In [14303]:
%%sql

CREATE OR REPLACE PROCEDURE add_variant(
    in_variant_name VARCHAR,
    in_attr_names    JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_variant_id     INTEGER;
    i                  INTEGER := 0;
    key_count          INTEGER;
    attr_id            INTEGER;
    attr_name          VARCHAR;
    variant_attr_name  VARCHAR;
    element            JSON;
    element_type       TEXT;
BEGIN
    -- Check if the variant already exists. If so, use its id.
    SELECT id INTO new_variant_id
      FROM variant
     WHERE name = in_variant_name
     LIMIT 1;

    IF new_variant_id IS NULL THEN
        INSERT INTO variant(name)
        VALUES (in_variant_name)
        RETURNING id INTO new_variant_id;
    END IF;

    IF in_attr_names IS NOT NULL THEN
        key_count := json_array_length(in_attr_names::json);
        WHILE i < key_count LOOP
            element := in_attr_names::json -> i;
            element_type := json_typeof(element);

            IF element_type = 'array' THEN
                -- Expecting a two-element tuple: [attribute_name, variant_attr_name]
                attr_name := element ->> 0;
                variant_attr_name := element ->> 1;
            ELSE
                -- If a simple string is provided, use it for both attribute lookup and variant_attr name.
                attr_name := in_attr_names::json ->> i;
                variant_attr_name := attr_name;
            END IF;

            -- Look up the attribute id using the attribute name.
            SELECT id INTO attr_id
              FROM attribute
             WHERE name = attr_name
             LIMIT 1;

            IF attr_id IS NULL THEN
                RAISE EXCEPTION 'Attribute with name "%" not found', attr_name;
            END IF;

            -- Link the attribute to the variant.
            CALL link_attr_to_variant(new_variant_id, attr_id, variant_attr_name);
            i := i + 1;
        END LOOP;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [14304]:
%%sql

CREATE OR REPLACE FUNCTION get_variant(
  p_variant_name VARCHAR,
  p_max_depth INTEGER
)
RETURNS TABLE (
  depth       INTEGER,
  address     TEXT,
  activates   JSON,
  effected_by JSON
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE
    root_variant AS (
      SELECT id 
      FROM variant 
      WHERE name = p_variant_name
    ),
    rec AS (
      -- Base step:
      -- For each variant_attr in the root variant,
      -- start with an empty finalized prefix and the attribute name as pending.
      SELECT 
         va.id AS variant_attr_id,
         va.attribute_id,
         va.name AS variant_attr_name,
         va.causation_index,
         ARRAY[va.causation_index] AS flattened_path,
         '[]'::jsonb AS addr_prefix,
         va.name AS pending,
         va.variant_id AS current_variant_id,
         -va.id AS resolved_id,  -- simulation marker (negative means “not a real row”)
         NULL::INT AS subvariant_span_id,
         1 AS cur_depth,
         ARRAY[ARRAY[0, va.id]] AS resolved_path
      FROM variant_attr va
      WHERE va.variant_id = (SELECT id FROM root_variant)
      
      UNION ALL
      
      -- Recursive step:
      -- Look up a candidate span (that “activates” a subvariant) for the parent's attribute.
      -- Finalize the parent's pending attribute by pairing it with the candidate span label,
      -- then set the child attribute name as the new pending value.
      SELECT 
         child_va.id AS variant_attr_id,
         child_va.attribute_id,
         child_va.name AS variant_attr_name,
         child_va.causation_index,
         r.flattened_path || child_va.causation_index,
         -- Finalize parent's pending value:
         CASE 
           WHEN r.addr_prefix = '[]'::jsonb THEN jsonb_build_array(jsonb_build_array(r.pending, cand.label))
           ELSE r.addr_prefix || jsonb_build_array(jsonb_build_array(r.pending, cand.label))
         END AS addr_prefix,
         child_va.name AS pending,
         child_va.variant_id AS current_variant_id,
         -child_va.id AS resolved_id,
         NULL::INT AS subvariant_span_id,
         r.cur_depth + 1 AS cur_depth,
         r.resolved_path || ARRAY[ARRAY[cand.subvariant_span_id, child_va.id]]
      FROM rec r
      JOIN LATERAL (
         SELECT s.id AS span_id,
                ss.id AS subvariant_span_id,
                s.label,
                ss.variant_id
         FROM span s
         JOIN subvariant_span ss ON ss.span_id = s.id
         WHERE s.attribute_id = (
                   SELECT attribute_id 
                   FROM variant_attr 
                   WHERE id = r.variant_attr_id
               )
         ORDER BY ss.id
         LIMIT 1
      ) cand ON true
      JOIN variant_attr child_va ON child_va.variant_id = cand.variant_id
      WHERE r.cur_depth < p_max_depth
    ),
    final AS (
      -- Finalize the address by combining the prefix and pending.
      SELECT 
         rec.flattened_path,
         rec.cur_depth,
         rec.addr_prefix,
         rec.pending,
         rec.resolved_path
      FROM rec
    )
  SELECT 
    final.cur_depth AS depth,
    -- If no steps were finalized, output a flat array; otherwise output the prefix plus pending.
    CASE 
      WHEN final.addr_prefix = '[]'::jsonb THEN jsonb_build_array(final.pending)
      ELSE final.addr_prefix || jsonb_build_array(final.pending)
    END::text AS address,
    (
      SELECT json_agg(eff.name ORDER BY eff.name)
      FROM get_effects_from_resolved_path(final.resolved_path) eff
      WHERE eff.relationship = 'activates'
    ) AS activates,
    (
      SELECT json_agg(eff.name ORDER BY eff.name)
      FROM get_effects_from_resolved_path(final.resolved_path) eff
      WHERE eff.relationship = 'effected_by'
    ) AS effected_by
  FROM final
  ORDER BY final.flattened_path;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [14305]:
%%sql

CREATE OR REPLACE PROCEDURE delete_variant(p_variant_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM variant WHERE name = p_variant_name;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Entity

In [14306]:
%%sql

CREATE OR REPLACE PROCEDURE add_entity(
    in_name VARCHAR(255),
    in_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO entity(name, variant_id)
    VALUES (in_name, in_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Discrete Variant Attribute Value

In [14307]:
%%sql

CREATE OR REPLACE FUNCTION roll_discrete_varattr(
    p_possible_attr_id INTEGER,
    p_entity_state_id INTEGER,
    p_exclude_span_id INTEGER
) 
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_span_id INTEGER;
    v_max_double CONSTANT double precision := 1e308;
    v_variant_attr_id INTEGER;
    v_subvariant_span_id INTEGER;
BEGIN
    -- Retrieve the underlying variant attribute and associated span trigger.
    SELECT variant_attr_id, subvariant_span_id
      INTO v_variant_attr_id, v_subvariant_span_id
      FROM possible_attr
     WHERE id = p_possible_attr_id;
    
    WITH
    BaseSpans AS (
          SELECT s.id AS span_id, 
               va.id AS variant_attr_id, 
               NULL::INTEGER AS subvariant_span_id,
               s.weight AS base_weight
          FROM variant_attr va
          JOIN span s 
            ON s.attribute_id = va.attribute_id
          WHERE va.id = v_variant_attr_id
            AND s.type = 'discrete'
            AND (p_exclude_span_id = 0 OR s.id <> p_exclude_span_id)
            -- Exclude spans that were added for an effect
            AND s.id NOT IN (
                 SELECT ase.span_id
                   FROM add_span_eft ase
            )
     ),
    ActiveEffects AS (
         /*
           Join to the effect table. We only want effects whose
           triggering span (effect.span_id) has already been rolled into
           the current entity_state.
         */
         SELECT e.id AS effect_id, 
                e.to_modify_possible_attr_id,
                e.activating_possible_attr_id
         FROM effect e
         WHERE e.variant_id = (
                 SELECT variant_id
                   FROM possible_attr
                  WHERE id = p_possible_attr_id
             )
           AND e.span_id IN (
                 SELECT span_id
                   FROM attr_value
                  WHERE entity_state_id = p_entity_state_id
             )
           AND e.to_modify_possible_attr_id IN (
                 SELECT pa.id
                   FROM possible_attr pa
                  WHERE pa.variant_attr_id = v_variant_attr_id
                    AND same_slot(pa.id, p_possible_attr_id)
             )
    ),
    InactiveSpans AS (
         SELECT DISTINCT rse.span_id
         FROM remove_span_eft rse
         JOIN ActiveEffects ae ON ae.effect_id = rse.effect_id
    ), 
    ActivatedSpans AS (
         SELECT DISTINCT 
                ase.span_id, 
                ae.to_modify_possible_attr_id AS variant_attr_id, 
                NULL::INTEGER AS subvariant_span_id, 
                0.0 AS base_weight
         FROM add_span_eft ase
         JOIN ActiveEffects ae ON ae.effect_id = ase.effect_id
    ), 
    DeltaWeights AS (
         SELECT dwe.span_id, 
                SUM(dwe.delta_weight) AS total_delta
         FROM delta_weight_eft dwe
         JOIN ActiveEffects ae ON ae.effect_id = dwe.effect_id
         GROUP BY dwe.span_id
    ), 
    AllRelevantSpans AS (
         SELECT b.span_id, b.variant_attr_id, b.subvariant_span_id, b.base_weight
         FROM BaseSpans b
         WHERE b.span_id NOT IN (SELECT span_id FROM InactiveSpans)
         UNION
         SELECT a.span_id, a.variant_attr_id, a.subvariant_span_id, a.base_weight
         FROM ActivatedSpans a
         WHERE a.span_id NOT IN (SELECT span_id FROM InactiveSpans)
    ), 
    FinalSpans AS (
         SELECT ars.span_id, 
                ars.variant_attr_id, 
                ars.subvariant_span_id,
                COALESCE(ars.base_weight, 0) + COALESCE(dw.total_delta, 0) AS effective_weight
         FROM AllRelevantSpans ars
         LEFT JOIN DeltaWeights dw ON dw.span_id = ars.span_id
    ),
    SpanCounts AS (
         SELECT ns.num_spans, 
                SUM(fs.effective_weight / ns.num_spans) AS total_weight
         FROM FinalSpans fs
         CROSS JOIN (
             SELECT COUNT(*)::double precision AS num_spans 
             FROM FinalSpans 
             WHERE effective_weight > 0
         ) ns
         WHERE fs.effective_weight > 0
         GROUP BY ns.num_spans
    ),
    AdjustedSpans AS (
         SELECT fs.span_id,
                fs.variant_attr_id,
                fs.subvariant_span_id,
                fs.effective_weight,
                sc.num_spans,
                sc.total_weight,
                CASE 
                  WHEN fs.effective_weight < 1 
                    THEN ceil((fs.effective_weight * v_max_double) / (sc.total_weight * sc.num_spans))
                  ELSE ceil((fs.effective_weight / (sc.total_weight * sc.num_spans)) * v_max_double)
                END AS adjusted_weight
         FROM FinalSpans fs
         CROSS JOIN SpanCounts sc
         WHERE fs.effective_weight > 0
    ),
    Running AS (
         SELECT span_id,
                variant_attr_id,
                subvariant_span_id,
                adjusted_weight,
                SUM(adjusted_weight) OVER (ORDER BY span_id) AS running_total
         FROM AdjustedSpans
    ),
    OrderedSpansCTE AS (
         SELECT span_id,
                variant_attr_id,
                subvariant_span_id,
                adjusted_weight AS contextual_weight,
                running_total,
                LAG(running_total, 1, 0) OVER (ORDER BY span_id) AS prev_running_total
         FROM Running
    ),
    TotalAdjusted AS (
         SELECT MAX(running_total) AS v_total_weight
         FROM OrderedSpansCTE
    ),
    Rng AS (
         SELECT t.v_total_weight,
                floor(random() * t.v_total_weight) AS v_random_pick
         FROM TotalAdjusted t
    )
    SELECT o.span_id
      INTO v_span_id
      FROM OrderedSpansCTE o, Rng
      WHERE Rng.v_random_pick >= o.prev_running_total
        AND Rng.v_random_pick < o.running_total
      LIMIT 1;

    RETURN v_span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Gamma Range

In [14308]:
%%sql

CREATE OR REPLACE FUNCTION gamma_rng(shape double precision)
RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    d double precision;
    c double precision;
    x double precision;
    v double precision;
    u double precision;
BEGIN
    IF shape < 1 THEN
        -- Use the transformation: Gamma(shape) = Gamma(shape+1)*U^(1/shape)
        RETURN gamma_rng(shape + 1) * power(random(), 1.0 / shape);
    END IF;
    d := shape - 1.0/3.0;
    c := 1.0 / sqrt(9.0 * d);
    LOOP
        -- Generate a standard normal via Box–Muller:
        x := sqrt(-2 * ln(random())) * cos(2 * 3.141592653589793 * random());
        v := 1 + c * x;
        IF v <= 0 THEN
            CONTINUE;
        END IF;
        v := v * v * v;
        u := random();
        IF u < 1 - 0.0331 * x * x * x * x THEN
            RETURN d * v;
        END IF;
        IF ln(u) < 0.5 * x * x + d * (1 - v + ln(v)) THEN
            RETURN d * v;
        END IF;
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Weighted Random Number

In [14309]:
%%sql

-- using a fast Marsaglia–Tsang method
CREATE OR REPLACE FUNCTION weighted_random(
    min_val double precision,                   -- Minimum of desired range.
    max_val double precision,                   -- Maximum of desired range (must be > min_val).
    mode_val double precision,                  -- Desired mode (peak) in the same units.
    skew double precision DEFAULT 0,            -- Skew adjustment; between -1 and 1 only, 0 = no skew.
    concentration double precision DEFAULT 4    -- Concentration (k); should be >2 for a proper unimodal shape.
) RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    m_norm      double precision;  -- Normalized desired mode in [0,1].
    alpha       double precision;
    beta        double precision;
    x_val       double precision;
    y_val       double precision;
    beta_sample double precision;
BEGIN
    IF min_val >= max_val THEN
        RAISE EXCEPTION 'min_val must be less than max_val';
    END IF;
    IF concentration < 0 THEN
        RAISE EXCEPTION 'concentration must be non-negative';
    END IF;
    IF concentration <= 2 THEN
        -- When concentration <= 2 the Beta isn't unimodal; fallback to uniform.
        RETURN min_val + random() * (max_val - min_val);
    END IF;

    -- Normalize the desired mode to [0,1].
    m_norm := (mode_val - min_val) / (max_val - min_val);

    -- Parameterize α and β so that (α-1)/(α+β-2) equals m_norm.
    alpha := m_norm * (concentration - 2) + 1;
    beta  := (1 - m_norm) * (concentration - 2) + 1;

    -- Apply skew correction if desired.
    IF skew <> 0 THEN
        alpha := alpha * (1 + skew);
        beta  := beta  * (1 - skew);
    END IF;

    -- Generate a Beta(α,β) random variable via the Gamma method.
    x_val := gamma_rng(alpha);
    y_val := gamma_rng(beta);
    beta_sample := x_val / (x_val + y_val);

    -- Scale the [0,1] Beta sample to [min_val, max_val].
    RETURN min_val + beta_sample * (max_val - min_val);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Continuous Variant Attribute Value

In [14310]:
%%sql

CREATE OR REPLACE FUNCTION roll_continuous_varattr(
    p_possible_attr_id INTEGER,
    p_entity_state_id   INTEGER,
    p_exclude_span_id   INTEGER
)
RETURNS TABLE(chosen_span_id integer, chosen_value double precision)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id         INTEGER;
    v_decimals             INTEGER;
    v_min                  DOUBLE PRECISION;
    v_max                  DOUBLE PRECISION;
    v_mode                 DOUBLE PRECISION;
    v_concentration        DOUBLE PRECISION;
    v_skew                 DOUBLE PRECISION;
    v_total_delta_mode     DOUBLE PRECISION := 0;
    v_total_delta_conc     DOUBLE PRECISION := 0;
    v_total_delta_skew     DOUBLE PRECISION := 0;
    v_eff_min              DOUBLE PRECISION;
    v_eff_max              DOUBLE PRECISION;
    v_eff_mode             DOUBLE PRECISION;
    v_result               DOUBLE PRECISION;
    v_clamped_result       DOUBLE PRECISION;
    v_chosen_span_id       INTEGER;
    dummy_eff_min          DOUBLE PRECISION;
    dummy_eff_max          DOUBLE PRECISION;
    v_variant_attr_id INTEGER;
BEGIN
    -- Retrieve attribute details from the possible_attr record.
    SELECT va.attribute_id, a.decimals, a.min_value, a.max_value, a.mode_value, 
           a.concentration, a.skew, va.id
      INTO v_attribute_id, v_decimals, v_min, v_max, v_mode, v_concentration, v_skew, v_variant_attr_id
      FROM possible_attr pa
      JOIN variant_attr va ON va.id = pa.variant_attr_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE pa.id = p_possible_attr_id
     LIMIT 1;

    IF v_attribute_id IS NULL THEN
        RAISE EXCEPTION 'No matching attribute found for possible_attr_id %', p_possible_attr_id;
    END IF;

    -- Sum continuous attribute delta values from effects that have been triggered.
    SELECT COALESCE(SUM(dpe.delta_mode), 0),
           COALESCE(SUM(dpe.delta_conc), 0),
           COALESCE(SUM(dpe.delta_skew), 0)
      INTO v_total_delta_mode, v_total_delta_conc, v_total_delta_skew
      FROM delta_param_eft dpe
      JOIN effect e ON e.id = dpe.effect_id
      JOIN possible_attr pa ON pa.id = e.to_modify_possible_attr_id
     WHERE pa.variant_attr_id = v_variant_attr_id
       AND e.variant_id = (
             SELECT variant_id FROM possible_attr WHERE id = p_possible_attr_id
       )
       AND e.span_id IN (
             SELECT span_id FROM attr_value WHERE entity_state_id = p_entity_state_id
       )
       AND same_slot(pa.id, p_possible_attr_id);

    -- Compute effective min, max, and mode.
    IF (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0) THEN
        v_eff_min  := (v_min + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_max  := (v_max + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_mode := (v_mode + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
    ELSE
        v_eff_min  := v_min + v_total_delta_mode;
        v_eff_max  := v_max + v_total_delta_mode;
        v_eff_mode := v_mode + v_total_delta_mode;
    END IF;

    IF v_eff_max < v_eff_min THEN
        -- Swap if needed.
        v_result  := v_eff_min;
        v_eff_min := v_eff_max;
        v_eff_max := v_result;
    END IF;

    -- Generate a weighted random value.
    v_result := weighted_random(
                   v_eff_min, 
                   v_eff_max, 
                   v_eff_mode, 
                   COALESCE(v_skew, 0),
                   COALESCE(v_concentration, 0)
               );
    v_result := round(v_result::numeric, v_decimals)::double precision;

    -- Clamp the result to the effective range.
    IF v_result < v_eff_min THEN
        v_clamped_result := v_eff_min;
    ELSIF v_result > v_eff_max THEN
        v_clamped_result := v_eff_max;
    ELSE
        v_clamped_result := v_result;
    END IF;

    -- Select the span matching the computed result.
    -- We join from span -> subvariant_span -> possible_attr and use same_slot so that any 
    -- possible_attr row structurally equivalent to p_possible_attr_id is used.
    IF p_exclude_span_id IS NOT NULL AND p_exclude_span_id <> 0 THEN
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN subvariant_span ss ON ss.span_id = s.id
          JOIN possible_attr pa ON pa.subvariant_span_id = ss.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND s.id <> p_exclude_span_id
           AND same_slot(pa.id, p_possible_attr_id)
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    ELSE
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN subvariant_span ss ON ss.span_id = s.id
          JOIN possible_attr pa ON pa.subvariant_span_id = ss.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND same_slot(pa.id, p_possible_attr_id)
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    END IF;

    chosen_span_id := v_chosen_span_id;
    chosen_value   := v_clamped_result;
    RETURN NEXT;
    RETURN;
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity State

In [14311]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_state(
    IN p_entity_name VARCHAR,
    IN p_time        DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_id               INTEGER;
    v_root_variant_id         INTEGER;
    v_entity_state_id         INTEGER;
    v_is_regenerate           BOOLEAN;
    v_current_variant         INTEGER;
    v_trigger_ss              INTEGER;  -- the subvariant_span that triggered this variant (NULL for top-level)
    v_parent_possible_attr_id INTEGER;  -- parent's possible_attr id (NULL for top-level)
    cur_va_id                 INTEGER;
    cur_attr_type             TEXT;
    v_subvariant_span_id      INTEGER;
    v_existing_attr_val_id    INTEGER;
    v_existing_span_id        INTEGER;
    v_lock_count              INTEGER := 0;
    v_new_span_id             INTEGER;
    v_new_numeric             DOUBLE PRECISION;
    v_used_span_id            INTEGER;
    v_sub_variant_id          INTEGER;
    v_attr_counter            INTEGER := 0;
    v_queue_id                INTEGER;  -- for stack (LIFO) ordering
    cur_possible_attr_id      INTEGER;  -- resolved unique possible_attr id for this combination
    rec                       RECORD;
BEGIN
    -- Look up the entity by name.
    SELECT id, variant_id
      INTO v_entity_id, v_root_variant_id
      FROM entity
     WHERE name = p_entity_name;
    IF v_entity_id IS NULL THEN
       RAISE EXCEPTION 'No entity found with name=%', p_entity_name;
    END IF;

    -- Check if an entity_state already exists for this entity at the given time.
    SELECT id 
      INTO v_entity_state_id
      FROM entity_state
     WHERE entity_id = v_entity_id
       AND "time" = p_time
     LIMIT 1;
    IF v_entity_state_id IS NOT NULL THEN
        v_is_regenerate := TRUE;
    ELSE
        INSERT INTO entity_state(entity_id, "time")
        VALUES (v_entity_id, p_time)
        RETURNING id INTO v_entity_state_id;
        v_is_regenerate := FALSE;
    END IF;

    -- Create a temporary queue that holds variants to process.
    -- Each queue row holds:
    --   variant_id: the variant whose attributes will be generated.
    --   subvariant_span_id: the span that triggered this variant (NULL for top-level)
    --   parent_possible_attr_id: the parent's possible_attr id (NULL for top-level).
    DROP TABLE IF EXISTS _variant_queue;
    CREATE TEMPORARY TABLE _variant_queue (
        queue_id serial PRIMARY KEY,
        variant_id INTEGER,
        subvariant_span_id INTEGER,
        parent_possible_attr_id INTEGER
    ) ON COMMIT DROP;
    -- Top-level variant: no triggering span and no parent.
    INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id)
      VALUES (v_root_variant_id, NULL, NULL);

    -- Temporary table to hold the current variant's attributes.
    DROP TABLE IF EXISTS _variant_attrs;
    CREATE TEMPORARY TABLE _variant_attrs (
        va_id    INTEGER,
        attr_type TEXT
    ) ON COMMIT DROP;

    -- Process the variant queue (LIFO for depth-first traversal).
    LOOP
        SELECT queue_id, variant_id, subvariant_span_id, parent_possible_attr_id
          INTO v_queue_id, v_current_variant, v_trigger_ss, v_parent_possible_attr_id
          FROM _variant_queue
         ORDER BY queue_id DESC
         LIMIT 1;
        EXIT WHEN NOT FOUND;
        DELETE FROM _variant_queue WHERE queue_id = v_queue_id;

        -- When processing a variant from the queue, treat its attributes as top-level.
        -- In other words, ignore the triggering subvariant span.
        v_subvariant_span_id := NULL;

        -- Load the attributes for the current variant.
        TRUNCATE _variant_attrs;
        INSERT INTO _variant_attrs (va_id, attr_type)
          SELECT va.id, a.type
            FROM variant_attr va
            JOIN attribute a ON a.id = va.attribute_id
           WHERE va.variant_id = v_current_variant;

        v_attr_counter := 0;
        FOR rec IN
            SELECT va_id, attr_type FROM _variant_attrs
        LOOP
            cur_va_id     := rec.va_id;
            cur_attr_type := rec.attr_type;
            v_attr_counter := v_attr_counter + 1;

            -- Use v_subvariant_span_id (which is now always NULL) for this variant's own attributes.
            -- This forces a fresh resolution for possible_attr in the current variant.
            -- (The parent_possible_attr_id is still set from the queue row for subvariants.)
            
            -- Resolve (or create) the unique possible_attr record for this combination.
            -- Uniqueness is defined by (parent_possible_attr_id, subvariant_span_id, variant_attr_id).
            SELECT id
              INTO cur_possible_attr_id
              FROM possible_attr
             WHERE variant_attr_id = cur_va_id
               AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                    OR parent_possible_attr_id = v_parent_possible_attr_id)
               AND ((subvariant_span_id IS NULL AND v_subvariant_span_id IS NULL)
                    OR subvariant_span_id = v_subvariant_span_id)
             LIMIT 1;
            IF NOT FOUND THEN
                INSERT INTO possible_attr (variant_attr_id, subvariant_span_id, parent_possible_attr_id)
                VALUES (cur_va_id, v_subvariant_span_id, v_parent_possible_attr_id)
                RETURNING id INTO cur_possible_attr_id;
            END IF;

            -- Process or update the attr_value for this resolved possible_attr.
            IF v_is_regenerate THEN
                SELECT id, span_id
                  INTO v_existing_attr_val_id, v_existing_span_id
                  FROM attr_value
                 WHERE entity_state_id = v_entity_state_id
                   AND possible_attr_id = cur_possible_attr_id
                 LIMIT 1;
            ELSE
                v_existing_attr_val_id := NULL;
                v_existing_span_id := NULL;
            END IF;

            IF v_is_regenerate AND v_existing_attr_val_id IS NOT NULL THEN
                SELECT count(*) INTO v_lock_count
                  FROM attr_val_lock
                 WHERE locked_attr_val_id = v_existing_attr_val_id;
                IF v_lock_count > 0 THEN
                    v_used_span_id := v_existing_span_id;
                ELSE
                    IF cur_attr_type = 'discrete' THEN
                        v_new_span_id := roll_discrete_varattr(cur_possible_attr_id, v_entity_state_id, 0);
                        UPDATE attr_value
                           SET span_id = v_new_span_id
                         WHERE id = v_existing_attr_val_id;
                        v_used_span_id := v_new_span_id;
                    ELSE
                        SELECT t.chosen_span_id, t.chosen_value
                          INTO v_new_span_id, v_new_numeric
                          FROM roll_continuous_varattr(cur_possible_attr_id, v_entity_state_id, 0) t
                         LIMIT 1;
                        UPDATE attr_value
                           SET span_id = v_new_span_id,
                               numeric_value = v_new_numeric
                         WHERE id = v_existing_attr_val_id;
                        v_used_span_id := v_new_span_id;
                    END IF;
                END IF;
            ELSE
                IF cur_attr_type = 'discrete' THEN
                    v_new_span_id := roll_discrete_varattr(cur_possible_attr_id, v_entity_state_id, 0);
                    INSERT INTO attr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        possible_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        NULL,
                        v_new_span_id,
                        cur_possible_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                ELSE
                    SELECT t.chosen_span_id, t.chosen_value
                      INTO v_new_span_id, v_new_numeric
                      FROM roll_continuous_varattr(cur_possible_attr_id, v_entity_state_id, 0) t
                     LIMIT 1;
                    INSERT INTO attr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        possible_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        v_new_numeric,
                        v_new_span_id,
                        cur_possible_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                END IF;
            END IF;

            -- If the chosen span activates a sub–variant, enqueue that variant.
            -- The sub–variant is defined by the subvariant_span that produced the used span.
            -- The new queue row carries the current possible_attr as the parent.
            SELECT variant_id
            INTO v_sub_variant_id
            FROM subvariant_span
            WHERE span_id = v_used_span_id  -- Fixed: use span_id column instead of id
            LIMIT 1;
            IF v_sub_variant_id IS NOT NULL THEN
                INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id)
                VALUES (v_sub_variant_id, v_used_span_id, cur_possible_attr_id)
                ON CONFLICT DO NOTHING;
            END IF;
        END LOOP;
    END LOOP;

    DROP TABLE IF EXISTS _variant_attrs;
    DROP TABLE IF EXISTS _variant_queue;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity State

In [14312]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_state(
    p_entity_name VARCHAR,
    p_time        DOUBLE PRECISION
)
RETURNS TABLE (
    "index"       BIGINT,
    "address"     TEXT,
    activates     JSON,
    effected_by   JSON,
    numeric_value DOUBLE PRECISION,
    span          VARCHAR,
    locked        BOOLEAN
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE entity_info AS (
      -- Retrieve the entity_state (by matching the entity name and time) and its variant.
      SELECT es.id AS entity_state_id,
             e.variant_id
      FROM entity_state es
      JOIN entity e ON e.id = es.entity_id
      WHERE e.name = p_entity_name
        AND es."time" = p_time
  ),
  base AS (
      -- Base step: select all top-level possible_attr records (those with no parent)
      -- for the variant corresponding to the entity state.
      SELECT 
          pa.id AS possible_attr_id,
          va.name AS attribute_name,
          va.causation_index,
          ARRAY[va.causation_index] AS flattened_path,
          jsonb_build_array(
             CASE 
               WHEN s.label IS NOT NULL 
                 THEN jsonb_build_array(va.name, s.label)
               ELSE jsonb_build_array(va.name)
             END
          ) AS agg_label,
          av.id AS attr_val_id,
          av.numeric_value,
          av.span_id
      FROM possible_attr pa
      JOIN variant_attr va 
        ON va.id = pa.variant_attr_id
      JOIN entity_info ei 
        ON va.variant_id = ei.variant_id
      JOIN attr_value av 
        ON av.possible_attr_id = pa.id 
       AND av.entity_state_id = ei.entity_state_id
      LEFT JOIN subvariant_span ss 
        ON ss.id = pa.subvariant_span_id
      LEFT JOIN span s 
        ON s.id = ss.span_id
      WHERE pa.parent_possible_attr_id IS NULL
  ),
  recursive_flattened AS (
      -- Start with the base nodes…
      SELECT * FROM base
      UNION ALL
      -- …and then for each node, recursively join its children.
      SELECT 
          child.possible_attr_id,
          child.attribute_name,
          child.causation_index,
          rf.flattened_path || child.causation_index AS flattened_path,
          rf.agg_label || jsonb_build_array(
             CASE 
               WHEN child_span.label IS NOT NULL 
                 THEN jsonb_build_array(child.attribute_name, child_span.label)
               ELSE jsonb_build_array(child.attribute_name)
             END
          ) AS agg_label,
          child.attr_val_id,
          child.numeric_value,
          child.span_id
      FROM (
          -- Get child nodes: all possible_attr records that have a non-null parent.
          SELECT 
              pa.id AS possible_attr_id,
              va.name AS attribute_name,
              va.causation_index,
              pa.parent_possible_attr_id,
              av.id AS attr_val_id,
              av.numeric_value,
              av.span_id,
              pa.subvariant_span_id
          FROM possible_attr pa
          JOIN variant_attr va 
            ON va.id = pa.variant_attr_id
          JOIN attr_value av 
            ON av.possible_attr_id = pa.id 
           AND av.entity_state_id = (SELECT entity_state_id FROM entity_info)
          WHERE pa.parent_possible_attr_id IS NOT NULL
      ) child
      JOIN recursive_flattened rf 
         ON child.parent_possible_attr_id = rf.possible_attr_id
      LEFT JOIN subvariant_span ss_child 
         ON ss_child.id = child.subvariant_span_id
      LEFT JOIN span child_span 
         ON child_span.id = ss_child.span_id
  ),
  ordered AS (
      -- Order the flattened nodes by the flattened_path and assign a sequential overall_index.
      SELECT 
          row_number() OVER (ORDER BY rf.flattened_path) AS overall_index,
          rf.agg_label,
          rf.possible_attr_id AS resolved_id,
          rf.flattened_path,
          rf.attr_val_id,
          rf.numeric_value,
          rf.span_id
      FROM recursive_flattened rf
  )
  SELECT 
      o.overall_index AS "index",
      o.agg_label::text AS "address",
      (
         -- Activating effects: only those for the rolled span and matching the possibility slot.
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.variant_id = (SELECT variant_id FROM entity_info)
           AND e.span_id = o.span_id
           AND same_slot(e.activating_possible_attr_id, o.resolved_id)
      ) AS activates,
      (
         -- Modifying effects: only include effects whose to_modify slot matches this row
         -- and whose activating effect (the triggering span) has been rolled in an earlier row.
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.variant_id = (SELECT variant_id FROM entity_info)
           AND e.span_id = o.span_id
           AND same_slot(e.to_modify_possible_attr_id, o.resolved_id)
           AND EXISTS (
                SELECT 1
                FROM ordered o2
                WHERE o2.overall_index < o.overall_index
                  AND same_slot(o2.resolved_id, e.activating_possible_attr_id)
           )
      ) AS effected_by,
      o.numeric_value,
      (
         SELECT s.label
         FROM span s
         WHERE s.id = o.span_id
      ) AS span,
      EXISTS (
         SELECT 1 FROM attr_val_lock l 
         WHERE l.locked_attr_val_id = o.attr_val_id
      ) AS locked
  FROM ordered o
  ORDER BY o.overall_index;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity Group

In [14313]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_group(
    p_variant_name         VARCHAR,
    p_group_name           VARCHAR,
    p_entity_name_template VARCHAR,
    p_num_entities         INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id       INTEGER;
    v_entity_group_id  INTEGER;
    v_entity_id        INTEGER;
    v_idx              INTEGER;
    v_entity_name      VARCHAR(255);
BEGIN
    -- Look up the variant id using the variant name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF v_variant_id IS NULL THEN
        RAISE EXCEPTION 'No variant found with name=%', p_variant_name;
    END IF;

    -- Create a new entity group.
    INSERT INTO entity_group (name)
    VALUES (p_group_name)
    RETURNING id INTO v_entity_group_id;

    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);

        -- Create a new entity using the looked-up variant id.
        INSERT INTO entity (variant_id, name)
        VALUES (v_variant_id, v_entity_name)
        RETURNING id INTO v_entity_id;

        INSERT INTO entity_group_link (entity_id, entity_group_id)
        VALUES (v_entity_id, v_entity_group_id);

        -- Call generate_entity_state using the new entity name and time (0 in this example).
        CALL generate_entity_state(v_entity_name, 0);
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delete Entity Group

In [14314]:
%%sql

CREATE OR REPLACE PROCEDURE delete_entity_group(
    p_group_name      VARCHAR,
    p_delete_entities BOOLEAN DEFAULT false
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_group_id INTEGER;
BEGIN
    -- Retrieve the entity group id based on the given name.
    SELECT id INTO v_entity_group_id
    FROM entity_group
    WHERE name = p_group_name;

    IF NOT FOUND THEN
        RAISE NOTICE 'Entity group "%" not found.', p_group_name;
        RETURN;
    END IF;

    -- If the flag is true, delete all entities that are linked to this group.
    IF p_delete_entities THEN
        DELETE FROM entity
        WHERE id IN (
            SELECT entity_id
            FROM entity_group_link
            WHERE entity_group_id = v_entity_group_id
        );
    END IF;

    -- Delete the entity group. The ON DELETE CASCADE on the foreign key
    -- in entity_group_link will automatically remove any associated links.
    DELETE FROM entity_group
    WHERE id = v_entity_group_id;

    RAISE NOTICE 'Entity group "%" deleted successfully.', p_group_name;
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity Group at Time

In [14315]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_group_at_time(
    p_group_name VARCHAR,
    p_time       DOUBLE PRECISION
)
RETURNS TABLE (
    entity_name      VARCHAR,
    entity_state_id  INTEGER,
    address          VARCHAR,
    numeric_value    DOUBLE PRECISION,
    span             VARCHAR,
    activates        VARCHAR,
    effected_by      VARCHAR,
    locked           BOOLEAN
)
LANGUAGE sql
AS $$
WITH eg AS (
    -- Find the entity group by its unique name.
    SELECT id
    FROM entity_group
    WHERE name = p_group_name
),
ent AS (
    -- Get all entities in the group.
    SELECT e.id, e.name
    FROM entity e
    JOIN entity_group_link egl ON e.id = egl.entity_id
    WHERE egl.entity_group_id = (SELECT id FROM eg)
),
est AS (
    -- For each entity, get its entity_state at the given time.
    SELECT e.id AS entity_id, e.name AS entity_name, es.id AS entity_state_id
    FROM ent e
    JOIN entity_state es ON es.entity_id = e.id
    WHERE es."time" = p_time
)
-- For each entity state, call get_entity_state and return one row per state detail.
SELECT
    est.entity_name,
    est.entity_state_id,
    vs.address,
    vs.numeric_value,
    vs.span,
    vs.activates,
    vs.effected_by,
    vs.locked
FROM est
CROSS JOIN LATERAL (
    SELECT *
    FROM get_entity_state(est.entity_name, p_time)
) AS vs;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Set Span as Subvariant

In [14316]:
%%sql

CREATE OR REPLACE PROCEDURE set_span_as_subvariant(
    p_attribute_name VARCHAR,
    p_span_label VARCHAR,
    p_variant_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_span_id INTEGER;
    v_variant_id INTEGER;
BEGIN
    -- Find the attribute by name.
    SELECT id
      INTO v_attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found.', p_attribute_name;
    END IF;

    -- Find the span by attribute and label.
    SELECT id
      INTO v_span_id
      FROM span
     WHERE attribute_id = v_attribute_id
       AND label = p_span_label;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span with label "%" for attribute "%" not found.', p_span_label, p_attribute_name;
    END IF;

    -- Try to find the variant by name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;

    -- If not found, create the variant.
    IF NOT FOUND THEN
        INSERT INTO variant (name)
             VALUES (p_variant_name)
         RETURNING id INTO v_variant_id;
    END IF;

    -- Insert the association into subvariant_span.
    -- If the association already exists, do nothing.
    INSERT INTO subvariant_span (span_id, variant_id)
         VALUES (v_span_id, v_variant_id)
    ON CONFLICT (variant_id, span_id) DO NOTHING;

    RAISE NOTICE 'Subvariant span set for attribute %, span %, variant %.',
                 p_attribute_name, p_span_label, p_variant_name;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Unset Span as Subvariant

In [14317]:
%%sql

CREATE OR REPLACE PROCEDURE unset_span_as_subvariant(
    p_attribute_name VARCHAR,
    p_span_label VARCHAR,
    p_variant_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_span_id INTEGER;
    v_variant_id INTEGER;
    v_deleted_count INTEGER;
BEGIN
    -- Find the attribute by name
    SELECT id
      INTO v_attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found.', p_attribute_name;
    END IF;

    -- Find the span by attribute and label
    SELECT id
      INTO v_span_id
      FROM span
     WHERE attribute_id = v_attribute_id
       AND label = p_span_label;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span with label "%" for attribute "%" not found.', p_span_label, p_attribute_name;
    END IF;

    -- Find the variant by name
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found.', p_variant_name;
    END IF;

    -- Delete the association from subvariant_span.
    DELETE FROM subvariant_span
     WHERE span_id = v_span_id
       AND variant_id = v_variant_id;
    
    GET DIAGNOSTICS v_deleted_count = ROW_COUNT;
    
    IF v_deleted_count = 0 THEN
        RAISE NOTICE 'No subvariant span association existed for attribute %, span %, variant %.',
                     p_attribute_name, p_span_label, p_variant_name;
    ELSE
        RAISE NOTICE 'Subvariant span association removed for attribute %, span %, variant %.',
                     p_attribute_name, p_span_label, p_variant_name;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Resolve Possible Attribute Path

In [14318]:
%%sql

CREATE OR REPLACE FUNCTION resolve_possible_attr_path(
  p_variant_id INT,
  p_path JSON
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
  v_chain_length         INT := json_array_length(p_path);
  v_current_variant      INT := p_variant_id;
  v_possible_attr_id     INT;
  v_parent_possible_attr INT;
  v_variant_attr_id      INT;
  v_subvariant_span_id   INT;
  v_sub_variant          INT;
  v_i                    INT;
  v_step                 JSON;
  v_attr_name            TEXT;
  v_span_label           TEXT;
BEGIN
  IF v_chain_length = 0 THEN
    RAISE EXCEPTION 'Variant path is empty';
  END IF;

  -- Process the root tuple (index 0); ignore its span label.
  v_step := p_path->0;
  v_attr_name := v_step->>0;
  SELECT id
    INTO v_variant_attr_id
    FROM variant_attr
   WHERE variant_id = v_current_variant
     AND name = v_attr_name
   ORDER BY id
   LIMIT 1;
  IF v_variant_attr_id IS NULL THEN
    RAISE EXCEPTION 'Variant attribute "%" not found in variant % at path index 0', v_attr_name, v_current_variant;
  END IF;

  SELECT id
    INTO v_possible_attr_id
    FROM possible_attr
   WHERE variant_attr_id = v_variant_attr_id
     AND subvariant_span_id IS NULL
     AND parent_possible_attr_id IS NULL
   LIMIT 1;
  IF v_possible_attr_id IS NULL THEN
    INSERT INTO possible_attr (variant_attr_id, subvariant_span_id, parent_possible_attr_id)
    VALUES (v_variant_attr_id, NULL, NULL)
    RETURNING id INTO v_possible_attr_id;
  END IF;
  v_parent_possible_attr := v_possible_attr_id;

  -- For each subsequent node, use the previous tuple's span label to activate a subvariant.
  FOR v_i IN 0 .. v_chain_length - 2 LOOP
    -- Use tuple at index v_i to get the activating span label.
    v_step := p_path->v_i;
    v_span_label := v_step->>1;
    SELECT ss.id, ss.variant_id
      INTO v_subvariant_span_id, v_sub_variant
      FROM subvariant_span ss
      JOIN span s ON s.id = ss.span_id
     WHERE s.label = v_span_label
       AND s.attribute_id = (SELECT attribute_id FROM variant_attr WHERE id = v_variant_attr_id)
     ORDER BY ss.id
     LIMIT 1;
    IF v_subvariant_span_id IS NULL THEN
      RAISE EXCEPTION 'No span with label "%" found for variant attribute "%" at path index %', v_span_label, v_attr_name, v_i;
    END IF;
    
    v_current_variant := v_sub_variant;

    -- Process the next tuple's attribute (index v_i+1).
    v_step := p_path->(v_i + 1);
    v_attr_name := v_step->>0;
    SELECT id
      INTO v_variant_attr_id
      FROM variant_attr
     WHERE variant_id = v_current_variant
       AND name = v_attr_name
     ORDER BY id
     LIMIT 1;
    IF v_variant_attr_id IS NULL THEN
      RAISE EXCEPTION 'Variant attribute "%" not found in variant % at path index %', v_attr_name, v_current_variant, v_i + 1;
    END IF;

    SELECT id
      INTO v_possible_attr_id
      FROM possible_attr
     WHERE variant_attr_id = v_variant_attr_id
       AND subvariant_span_id = v_subvariant_span_id
       AND parent_possible_attr_id = v_parent_possible_attr
     LIMIT 1;
    IF v_possible_attr_id IS NULL THEN
      INSERT INTO possible_attr (variant_attr_id, subvariant_span_id, parent_possible_attr_id)
      VALUES (v_variant_attr_id, v_subvariant_span_id, v_parent_possible_attr)
      RETURNING id INTO v_possible_attr_id;
    END IF;
    v_parent_possible_attr := v_possible_attr_id;
  END LOOP;

  RETURN v_possible_attr_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Possible Attribute Chain

In [14319]:
%%sql

CREATE OR REPLACE FUNCTION get_possible_attr_chain(
    p_possible_attr_id INT
) RETURNS TABLE(level INT, variant_attr_id INT, causation_index INT)
LANGUAGE plpgsql
AS $$
BEGIN
RETURN QUERY WITH RECURSIVE chain(lev, vav_id, va_id, ci, parent) AS (
    -- Start at the leaf: assign level = 1.
    SELECT 1,
           v.id,
           v.variant_attr_id,
           va.causation_index,
           v.parent_possible_attr_id
      FROM possible_attr v
      JOIN variant_attr va ON va.id = v.variant_attr_id
     WHERE v.id = p_possible_attr_id
    UNION ALL
    -- Walk upward.
    SELECT c.lev + 1,
           v.id,
           v.variant_attr_id,
           va.causation_index,
           v.parent_possible_attr_id
      FROM possible_attr v
      JOIN variant_attr va ON va.id = v.variant_attr_id
      JOIN chain c ON v.id = c.parent
)
-- Reverse the order so that the top-most attribute is first.
SELECT row_number() OVER (ORDER BY lev DESC) AS level,
       va_id AS variant_attr_id,
       ci AS causation_index
  FROM chain
 ORDER BY level;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Compare Possible Attribute Order

In [14320]:
%%sql

CREATE OR REPLACE FUNCTION compare_possible_attr_order(
    p_a INT,
    p_b INT
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN
        SELECT COALESCE(a.level, b.level) AS level,
               a.causation_index AS a_ci,
               b.causation_index AS b_ci
          FROM get_possible_attr_chain(p_a) a
          FULL OUTER JOIN get_possible_attr_chain(p_b) b USING (level)
         ORDER BY level
    LOOP
        -- If one chain is shorter, treat the missing value as "less".
        IF rec.a_ci IS NULL AND rec.b_ci IS NOT NULL THEN
            RETURN -1;
        ELSIF rec.b_ci IS NULL AND rec.a_ci IS NOT NULL THEN
            RETURN 1;
        ELSIF rec.a_ci < rec.b_ci THEN
            RETURN -1;
        ELSIF rec.a_ci > rec.b_ci THEN
            RETURN 1;
        END IF;
        -- Otherwise they are equal at this level; continue.
    END LOOP;
    RETURN 0;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Effect

In [14321]:
%%sql

CREATE OR REPLACE PROCEDURE add_effect(
    p_variant_id              INT,
    p_effect_name             VARCHAR,
    p_triggering_span_label   VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_triggering_span_id         INT;
BEGIN
    -- Resolve the terminal nodes for each branch.
    v_activating_possible_attr_id := resolve_possible_attr_path(p_variant_id, p_activating_variant_path);
    v_to_modify_possible_attr_id  := resolve_possible_attr_path(p_variant_id, p_to_modify_variant_path);
    
    -- Look up the triggering span id using the provided label.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = p_triggering_span_label
        AND s.attribute_id = (
            SELECT va.attribute_id
              FROM variant_attr va
              JOIN possible_attr pa ON pa.variant_attr_id = va.id
             WHERE pa.id = v_activating_possible_attr_id
             LIMIT 1
        )
      LIMIT 1;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Triggering span with label "%" not found', p_triggering_span_label;
    END IF;
    
    -- Insert the new effect.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        p_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    );
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Variant ID

In [14322]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_id(
    p_variant_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    variant_id INTEGER;
BEGIN
    SELECT id
      INTO variant_id
      FROM variant
     WHERE name = p_variant_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found', variant_id;
    END IF;
    
    RETURN variant_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Attribute ID

In [14323]:
%%sql

CREATE OR REPLACE FUNCTION get_attribute_id(
    p_attribute_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    attribute_id INTEGER;
BEGIN
    SELECT id
      INTO attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found', p_attribute_name;
    END IF;
    
    RETURN attribute_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Span ID

In [14324]:
%%sql

CREATE OR REPLACE FUNCTION get_span_id(
    p_attribute_name VARCHAR,
    p_span_type      attr_type,
    p_span_label     VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    span_id INTEGER;
BEGIN
    SELECT s.id
      INTO span_id
      FROM span s
      JOIN attribute a ON a.id = s.attribute_id
     WHERE s.label = p_span_label
       AND a.name = p_attribute_name
       AND s.type = p_span_type;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span on attribute "%" of type "%" and label "%" not found', p_attribute_name, p_span_type, p_span_label;
    END IF;

    RETURN span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Effect ID

In [14325]:
%%sql

CREATE OR REPLACE FUNCTION get_effect_id(
    p_variant_name   VARCHAR,
    p_effect_name    VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    effect_id INTEGER;
BEGIN
    SELECT e.id
      INTO effect_id
      FROM effect e
      JOIN variant vt ON vt.id = e.variant_id
     WHERE vt.name = p_variant_name
       AND e.name = p_effect_name;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'effect "%" in variant "%" not found', p_effect_name, p_variant_name;
    END IF;

    RETURN effect_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Continuous Effect

In [14326]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_effect(
    p_effect_name             VARCHAR,
    p_variant_name            VARCHAR,
    p_activating_span_path    JSON,
    p_target_variant_attr_path JSON,
    p_delta_mode              DOUBLE PRECISION,
    p_delta_conc              DOUBLE PRECISION,
    p_delta_skew              DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                   INT;
    v_activating_possible_attr_id  INT;
    v_to_modify_possible_attr_id   INT;
    v_to_modify_attr_type          attr_type;
    v_triggering_span_id           INT;
    v_effect_id                    INT;
    v_triggering_span_label        VARCHAR;
    v_chain_length                 INT;
BEGIN
    -- Extract the triggering span label from the final tuple of the activating span path.
    v_chain_length := json_array_length(p_activating_span_path);
    IF v_chain_length = 0 THEN
         RAISE EXCEPTION 'Activating span is empty';
    END IF;
    -- For an array-of-arrays structure, get the second element (index 1) of the last tuple.
    v_triggering_span_label := (p_activating_span_path->(v_chain_length - 1))->>1;

    -- Resolve variant and possible attribute IDs using helper functions.
    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_possible_attr_path(v_variant_id, p_activating_span_path);
    v_to_modify_possible_attr_id  := resolve_possible_attr_path(v_variant_id, p_target_variant_attr_path);

    -- Ensure that the attribute type associated with v_to_modify_possible_attr_id is continuous.
    SELECT a.type
      INTO v_to_modify_attr_type
      FROM possible_attr pa
      JOIN variant_attr va ON va.id = pa.variant_attr_id
      JOIN attribute a ON a.id = va.attribute_id
      WHERE pa.id = v_to_modify_possible_attr_id
      LIMIT 1;

    IF v_to_modify_attr_type <> 'continuous' THEN
         RAISE EXCEPTION 'The attribute associated with to_modify_possible_attr_id (%) is %, but expected continuous',
                         v_to_modify_possible_attr_id, v_to_modify_attr_type;
    END IF;

    -- Resolve the triggering span ID using the extracted label.
    SELECT s.id
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = v_triggering_span_label
        AND s.attribute_id = (
              SELECT va.attribute_id
                FROM variant_attr va
                JOIN possible_attr pa ON pa.variant_attr_id = va.id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', v_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Insert the continuous effect details.
    INSERT INTO delta_param_eft(effect_id, delta_mode, delta_conc, delta_skew)
    VALUES (v_effect_id, p_delta_mode, p_delta_conc, p_delta_skew);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Discrete Effect

In [14327]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_effect(
    p_effect_name              VARCHAR,
    p_variant_name             VARCHAR,
    p_activating_span_path     JSON,
    p_target_variant_attr_path JSON,
    p_changes                  JSON,  -- Expecting an array of tuples: [[label, weight], ...]
    p_remove_unmentioned       BOOLEAN DEFAULT FALSE
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                  INT;
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_triggering_span_id          INT;
    v_effect_id                   INT;
    v_attribute_id                INT;
    v_attribute_name              VARCHAR;
    v_to_modify_attr_type         attr_type;  -- The type of the attribute to modify.
    v_span_id                     INT;
    rec_json                      JSON;       -- For iterating over p_changes JSON array.
    rec_record                    RECORD;     -- For iterating over the span query.
    v_label                       TEXT;
    v_weight                      DOUBLE PRECISION;
    -- Array to track labels processed in the p_changes array.
    v_processed_labels            TEXT[] := '{}';
    v_triggering_span_label       VARCHAR;
    v_chain_length                INT;
BEGIN
    -- Extract the triggering span label from the final tuple of the activating span path.
    v_chain_length := json_array_length(p_activating_span_path);
    IF v_chain_length = 0 THEN
         RAISE EXCEPTION 'Activating span path is empty';
    END IF;
    -- For an array-of-arrays structure, get the second element (index 1) of the last tuple.
    v_triggering_span_label := (p_activating_span_path->(v_chain_length - 1))->>1;

    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_possible_attr_path(v_variant_id, p_activating_span_path);
    v_to_modify_possible_attr_id  := resolve_possible_attr_path(v_variant_id, p_target_variant_attr_path);

    -- Resolve the triggering span ID using the extracted label.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = v_triggering_span_label
        AND s.attribute_id = (
              SELECT va.attribute_id
                FROM variant_attr va
                JOIN possible_attr pa ON pa.variant_attr_id = va.id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', v_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Retrieve the attribute associated with the to-modify branch, including its type.
    SELECT a.id, a.name, a.type
      INTO v_attribute_id, v_attribute_name, v_to_modify_attr_type
      FROM possible_attr pa
      JOIN variant_attr va ON va.id = pa.variant_attr_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE pa.id = v_to_modify_possible_attr_id
     LIMIT 1;

    -- Ensure that the attribute type is 'discrete'.
    IF v_to_modify_attr_type <> 'discrete' THEN
         RAISE EXCEPTION 'The attribute associated with to_modify_possible_attr_id (%) is %, but expected discrete',
                         v_to_modify_possible_attr_id, v_to_modify_attr_type;
    END IF;

    --------------------------------------------------------------------
    -- Process each change tuple in p_changes.
    -- For each tuple, record the label in v_processed_labels.
    --------------------------------------------------------------------
    IF p_changes IS NOT NULL THEN
       FOR rec_json IN SELECT * FROM json_array_elements(p_changes) LOOP
          -- Each tuple should be [label, weight].
          v_label  := rec_json->>0;
          v_weight := (rec_json->>1)::double precision;
          
          -- Record that this label was processed.
          v_processed_labels := array_append(v_processed_labels, v_label);

          SELECT id 
            INTO v_span_id
            FROM span
           WHERE attribute_id = v_attribute_id
             AND label = v_label
             AND type = 'discrete'
           LIMIT 1;

          IF NOT FOUND THEN
             -- Span does not exist: create it.
             INSERT INTO span(
                 attribute_id, 
                 label, 
                 type, 
                 is_pinned, 
                 weight, 
                 max_value, 
                 min_value
             )
             VALUES (v_attribute_id, v_label, 'discrete', false, v_weight, NULL, NULL)
             RETURNING id INTO v_span_id;

             -- Record that this span is activated by the effect.
             INSERT INTO add_span_eft(effect_id, span_id)
             VALUES (v_effect_id, v_span_id);
          ELSE
             IF v_weight = 0 THEN
                -- Mark the span as inactive for this effect.
                INSERT INTO remove_span_eft(effect_id, span_id)
                VALUES (v_effect_id, v_span_id);
             ELSE
                -- Record a delta–weight effect.
                INSERT INTO delta_weight_eft(effect_id, span_id, delta_weight)
                VALUES (v_effect_id, v_span_id, v_weight);
             END IF;
          END IF;
       END LOOP;
    END IF;

    --------------------------------------------------------------------
    -- If p_remove_unmentioned is true, remove all spans for the discrete
    -- attribute that were not mentioned in p_changes.
    --------------------------------------------------------------------
    IF p_remove_unmentioned THEN
       FOR rec_record IN
         SELECT id, label 
         FROM span
         WHERE attribute_id = v_attribute_id
           AND type = 'discrete'
           AND NOT (label = ANY(v_processed_labels))
       LOOP
         INSERT INTO remove_span_eft(effect_id, span_id)
         VALUES (v_effect_id, rec_record.id);
       END LOOP;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Remove Effect

In [14328]:
%%sql

CREATE OR REPLACE PROCEDURE remove_effect(
    p_effect_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_effect_id INT;
    v_deleted_count INT;
BEGIN
    -- Look up the effect by its unique name.
    SELECT id
      INTO v_effect_id
      FROM effect
     WHERE name = p_effect_name
     LIMIT 1;
     
    IF NOT FOUND THEN
       RAISE NOTICE 'Effect "%" not found. Nothing to remove.', p_effect_name;
       RETURN;
    END IF;

    -- Delete the effect record.
    DELETE FROM effect
     WHERE id = v_effect_id;
    
    GET DIAGNOSTICS v_deleted_count = ROW_COUNT;
    
    IF v_deleted_count > 0 THEN
       RAISE NOTICE 'Effect "%" removed successfully.', p_effect_name;
    ELSE
       RAISE NOTICE 'No effect was removed for "%".', p_effect_name;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Inspect Schema

## List Tables

In [14329]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

18 rows affected.

name
add_span_eft
attr_val_lock
attr_value
attribute
debug_log
delta_param_eft
delta_weight_eft
effect
entity
entity_group


## List Procedures

In [14330]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

20 rows affected.

name,arguments
add_cont_span_to_attr,"IN p_attribute_id integer, IN p_label character varying, IN p_new_min double precision, IN p_new_max double precision"
add_continuous_attribute,"IN p_name character varying, IN p_min_value double precision, IN p_mode_value double precision, IN p_max_value double precision, IN p_concentration double precision, IN p_skew double precision, IN p_decimals integer, IN p_units character varying, IN p_spans json DEFAULT NULL::json"
add_continuous_effect,"IN p_effect_name character varying, IN p_variant_name character varying, IN p_activating_span_path json, IN p_target_variant_attr_path json, IN p_delta_mode double precision, IN p_delta_conc double precision, IN p_delta_skew double precision"
add_disc_span_to_attr,"IN p_attribute_id integer, IN p_label character varying"
add_discrete_attribute,"IN in_name character varying, IN in_spans json"
add_discrete_effect,"IN p_effect_name character varying, IN p_variant_name character varying, IN p_activating_span_path json, IN p_target_variant_attr_path json, IN p_changes json, IN p_remove_unmentioned boolean DEFAULT false"
add_effect,"IN p_variant_id integer, IN p_effect_name character varying, IN p_triggering_span_label character varying, IN p_activating_variant_path json, IN p_to_modify_variant_path json"
add_entity,"IN in_name character varying, IN in_variant_id integer"
add_variant,"IN in_variant_name character varying, IN in_attr_names json"
delete_continuous_attribute,IN p_attribute_name character varying


## List Functions

In [14331]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

23 rows affected.

name,arguments,return_type
check_delete_possible_attr_from_attr_val,,trigger
check_delete_possible_attr_from_effect,,trigger
compare_possible_attr_order,"p_a integer, p_b integer",integer
debug_log,"p_procedure_name character varying, p_log_message text",void
delete_orphaned_span,,trigger
gamma_rng,shape double precision,double precision
get_attribute_id,p_attribute_name character varying,integer
get_continuous_attribute,p_attribute_name character varying,"TABLE(name character varying, type attr_type, min double precision, mode double precision, max double precision, concentration double precision, skew double precision, units character varying, decimals integer, span_label character varying, span_min double precision, span_max double precision)"
get_discrete_attribute,p_attribute_name character varying,"TABLE(attribute character varying, type attr_type, span_label character varying, span_weight double precision, is_pinned boolean)"
get_effect_id,"p_variant_name character varying, p_effect_name character varying",integer


## List Indices

In [14332]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

66 rows affected.

index_name,table_name
add_span_eft_pkey,add_span_eft
idx_add_span_eft_effect,add_span_eft
idx_add_span_eft_effid_spanid,add_span_eft
attr_val_lock_pkey,attr_val_lock
idx_attr_val_lock_locked,attr_val_lock
idx_attr_val_lock_locked_locking,attr_val_lock
idx_attr_val_lock_locking,attr_val_lock
unique_attr_val_lock,attr_val_lock
attr_value_pkey,attr_value
idx_attr_val_state_attr_span,attr_value


# Testing

## Gender Attribute

### Delete Gender Attribute

In [14333]:
%%sql

CALL delete_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Gender Attribute

In [14334]:
%%sql

call add_discrete_attribute('Gender', '[
    ["Male", 1], 
    ["Female", 1]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Gender Attribute

In [14335]:
%%sql

SELECT * FROM get_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

2 rows affected.

attribute,type,span_label,span_weight,is_pinned
Gender,discrete,Female,0.5,False
Gender,discrete,Male,0.5,False


## Hair Color Attribute

### Delete Hair Color Attribute

In [14336]:
%%sql

CALL delete_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Hair Color Attribute

In [14337]:
%%sql

call add_discrete_attribute('Hair Color', '[
    ["Blonde", 2], 
    ["Brown", 3],
    ["Golden", 2],
    ["Auburn", 2],
    ["Ginger", 1],
    ["Sandy", 2],
    ["Black", 3],
    ["Gray", 2]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Hair Color Attribute

In [14338]:
%%sql

SELECT * FROM get_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

8 rows affected.

attribute,type,span_label,span_weight,is_pinned
Hair Color,discrete,Ginger,0.058823529411764705,False
Hair Color,discrete,Auburn,0.11764705882352941,False
Hair Color,discrete,Blonde,0.11764705882352941,False
Hair Color,discrete,Golden,0.11764705882352941,False
Hair Color,discrete,Gray,0.11764705882352941,False
Hair Color,discrete,Sandy,0.11764705882352941,False
Hair Color,discrete,Black,0.17647058823529413,False
Hair Color,discrete,Brown,0.17647058823529413,False


## Height Attribute

### Delete Height Attribute

In [14339]:
%%sql

CALL delete_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Height Attribute

In [14340]:
%%sql

call add_continuous_attribute('Height', 0, 68, 96, 32, 0, 4, 'in.', '[
    ["Short", 60],
    ["Average", 74],
    ["Tall", 96]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Height Attribute

In [14341]:
%%sql

SELECT * FROM get_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

name,type,min,mode,max,concentration,skew,units,decimals,span_label,span_min,span_max
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Average,60.0,74.0
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Short,0.0,60.0
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Tall,74.0,96.0


## Human Variant

### Delete

In [14342]:
%%sql

CALL delete_variant('Human');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add

In [14343]:
%%sql

CALL add_variant('Human', '[
    "Gender",
    "Hair Color",
    "Height"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [14344]:
%%sql

SELECT * FROM get_variant('Human', 5);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

depth,address,activates,effected_by
1,"[""Gender""]",None,None
1,"[""Hair Color""]",None,None
1,"[""Height""]",None,None


## Generate a Group of Humans

### Delete

In [14345]:
%%sql

CALL delete_entity_group('Test Humans 1', true);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Create

In [14346]:
%%sql

CALL generate_entity_group('Human', 'Test Humans 1', 'Test Human %s', 10);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View

In [14347]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Humans 1', 0);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

30 rows affected.

entity_name,entity_state_id,address,numeric_value,span,activates,effected_by,locked
Test Human 1,1,"[[""Gender""]]",None,Female,[],[],False
Test Human 1,1,"[[""Hair Color""]]",None,Golden,[],[],False
Test Human 1,1,"[[""Height""]]",66.1023,None,[],[],False
Test Human 2,2,"[[""Gender""]]",None,Male,[],[],False
Test Human 2,2,"[[""Hair Color""]]",None,Gray,[],[],False
Test Human 2,2,"[[""Height""]]",74.7134,None,[],[],False
Test Human 3,3,"[[""Gender""]]",None,Female,[],[],False
Test Human 3,3,"[[""Hair Color""]]",None,Sandy,[],[],False
Test Human 3,3,"[[""Height""]]",71.6505,None,[],[],False
Test Human 4,4,"[[""Gender""]]",None,Female,[],[],False


# The Robot Test

## Paint Attribute

In [14348]:
%%sql

CALL add_discrete_attribute('Color', '[
        ["None", 3],
        ["Red", 2],
        ["Yellow", 2],
        ["Green", 2],
        ["Blue", 2],
        ["Orange", 2],
        ["White", 2],
        ["Black", 2],
        ["Purple", 2],
        ["Custom", 1, "Custom Pigment"]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14349]:
%%sql

CALL add_continuous_attribute('R', 0, 127, 255, 0, 0, 0, NULL, NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14350]:
%%sql

CALL add_continuous_attribute('G', 0, 127, 255, 0, 0, 0, NULL, NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14351]:
%%sql

CALL add_continuous_attribute('B', 0, 127, 255, 0, 0, 0, NULL, NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14352]:
%%sql

CALL add_variant('Custom Pigment', '["R", "G", "B"]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14353]:
%%sql

CALL add_discrete_attribute(
    'Sheen', '[
        ["Glossy", 3],
        ["Egg-Shell", 1],
        ["Matte", 2],
        ["Semi-Gloss", 2]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14354]:
%%sql

CALL add_variant('Paint', '["Color", "Sheen"]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14355]:
%%sql

CALL add_discrete_attribute('Paint','[["Paint", 1, "Paint"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Material Attribute

In [14356]:
%%sql

CALL add_discrete_attribute(
    'Material', '[
        ["Steel", 4],
        ["Aluminum", 6],
        ["Bronze", 2],
        ["Iron", 2],
        ["Carbon-Fiber", 6],
        ["PVC", 4],
        ["Animal Bone", 1],
        ["Wood", 2]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Power-Level Attribute

In [14357]:
%%sql

CALL add_continuous_attribute('Power-Level', 0, 50, 100, 24, 0, 2, NULL, '[
    ["Very Weak", 20],
    ["Weak", 40],
    ["Normal", 60],
    ["Strong", 80],
    ["Ultra", 100]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Length Attribute

In [14358]:
%%sql

CALL add_continuous_attribute('Length', .01, 36, 120, 36, 0, 2, 'in.', NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Diameter Attribute

In [14359]:
%%sql

CALL add_continuous_attribute('Diameter', .01, 1, 4, 12, 0, 2, 'in.', NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Sub-Arm Attribute

In [14360]:
%%sql

CALL add_discrete_attribute('Sub-Arm', '[["No", 4], ["Yes", 1, "Arm"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attachment Attribute

In [14361]:
%%sql

CALL add_discrete_attribute(
    'Attachment', '[
        ["Cutter", 3],
        ["Welder", 2],
        ["Pincher", 3],
        ["Grabber", 3],
        ["Laser", 1],
        ["Cannon", 1],
        ["Hammer", 2],
        ["Grappler", 1],
        ["Vacuum", 2]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Input Attribute

In [14362]:
%%sql

CALL add_discrete_attribute(
    'Input', '[
        ["Camera", 5],
        ["Microphone", 5],
        ["Smell-O-Sense", 2],
        ["Infra-Red", 3],
        ["X-Ray", 3],
        ["GPS", 4],
        ["Temporal", 1]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Output Attribute

In [14363]:
%%sql

CALL add_discrete_attribute(
    'Output', '[
        ["MIDI Tone", 5],
        ["Text Screen", 7],
        ["Clicker", 4],
        ["Recorded Phrases", 7],
        ["X-Ray", 5],
        ["Light Array", 5],
        ["Party Blower", 1],
        ["Kazoo", 1]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Foot Attribute

In [14364]:
%%sql

CALL add_discrete_attribute(
    'Foot', '[
        ["Roller Skate", 5],
        ["Stomper", 7],
        ["Tread", 4],
        ["Pogo", 7],
        ["Thruster", 5],
        ["Suction", 5]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Shape Attribute

In [14365]:
%%sql

CALL add_discrete_attribute(
    'Shape', '[
        ["Spherical", 5],
        ["Blocky", 7],
        ["Sculpted", 4],
        ["Spiky", 2],
        ["Segmented", 5],
        ["Donut", 1]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Power Source Attribute

In [14366]:
%%sql

CALL add_discrete_attribute(
    'Power Source', '[
        ["Battery", 15],
        ["Solar", 10],
        ["Diesel", 10],
        ["Wired", 12],
        ["Wind", 4],
        ["Nuclear Core", 4]
    ]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Neck Width Attribute

In [14367]:
%%sql

CALL add_continuous_attribute('Neck Width', 1, 10, 2, 2, 0, 1, 'in.', NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Weight Attribute

In [14368]:
%%sql

CALL add_continuous_attribute('Weight', .01, 20, 5000, 2, 0, 1, 'lbs.', NULL);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Torso Variant

In [14369]:
%%sql

CALL add_variant('Torso', '[
    "Shape", 
    "Material", 
    "Power-Level", 
    "Paint"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14370]:
%%sql

CALL add_discrete_attribute('Torso', '[["Torso", 1, "Torso"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Head Variant

In [14371]:
%%sql

CALL add_variant('Head', '[
    "Input", 
    "Output", 
    "Material", 
    "Power-Level", 
    "Paint"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14372]:
%%sql

CALL add_discrete_attribute('Head','[["Head", 1, "Head"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Leg Variant

In [14373]:
%%sql

CALL add_variant('Leg', '[
    "Foot", 
    "Material", 
    "Power-Level", 
    "Length",
    "Diameter",
    "Paint"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14374]:
%%sql

CALL add_discrete_attribute('Leg','[["Leg", 1, "Leg"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Hand Variant

In [14375]:
%%sql

-- Create the variant groupings
CALL add_variant('Hand', '[
    "Attachment", 
    "Material",  
    "Power-Level", 
    "Paint"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14376]:
%%sql

CALL add_discrete_attribute('Hand','[["Hand", 1, "Hand"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Arm Variant

In [14377]:
%%sql

CALL add_variant('Arm', '[
    "Hand", 
    "Material", 
    "Paint", 
    "Length", 
    "Diameter", 
    "Sub-Arm"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

In [14378]:
%%sql

CALL add_discrete_attribute('Arm','[["Arm", 1, "Arm"]]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Robot Variant

In [14379]:
%%sql

CALL add_variant('Robot', '[
    "Power Source",
    "Head",
    "Neck Width",
    "Torso",
    ["Arm", "Left Arm"], 
    ["Arm", "Right Arm"], 
    ["Leg", "Left Leg"], 
    ["Leg", "Right Leg"],
    "Weight"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Effects

### Welder hands cannot be made of wood'

In [14380]:
%%sql

-- Add effects

-- Test: discrete effect on same variant level, disable span
CALL add_discrete_effect(
    'Welder hands cannot be made of wood',
    'Hand',
    '[["Attachment", "Welder"]]',
    '[["Material"]]',
    '[["Wood", 0]]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Wooden hands have lower power level

In [14381]:
%%sql

-- Test: continous effect on same variant level
CALL add_continuous_effect(
    'Wooden hands have lower power level',
    'Hand',
    '[["Material", "Wood"]]',
    '[["Power-Level"]]',
    -20, 0, 0
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Spherical torsos are more likely to be painted white or blue

In [14382]:
%%sql

-- Test: stacked discrete effect on an attribute one level down
CALL add_discrete_effect(
    'Spherical torsos are more likely to be painted white or blue',
    'Torso',
    '[["Shape", "Spherical"]]',
    '[["Paint", "Paint"], ["Color"]]',
    '[["White", 10], ["Blue", 5]]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Animal bone torsos are more likely to be painted white or green

In [14383]:
%%sql

CALL add_discrete_effect(
    'Animal bone torsos are more likely to be painted white or green',
    'Torso',
    '[["Material", "Animal Bone"]]',
    '[["Paint", "Paint"], ["Color"]]',
    '[["White", 10], ["Green", 5]]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Wooden torsos have lower power level

In [14384]:
%%sql

-- Test: stacked continuous effect on same variant level
CALL add_continuous_effect(
    'Wooden torsos have lower power level',
    'Torso',
    '[["Material", "Wood"]]',
    '[["Power-Level"]]',
    -20, 0, 0
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Blocky torsos have higher power level

In [14385]:
%%sql

CALL add_continuous_effect(
    'Blocky torsos have higher power level',
    'Torso',
    '[["Shape", "Blocky"]]',
    '[["Power-Level"]]',
    15, 0, 0
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### All diesel-powered robots have a roller skate on their left foot

In [14386]:
%%sql

-- Test: discrete effect on an attribute one level down, restricting options
CALL add_discrete_effect(
    'All diesel-powered robots have a roller skate on their left foot',
    'Robot',
    '[["Power Source", "Diesel"]]',
    '[["Left Leg", "Leg"], ["Foot"]]',	
    '[["Roller Skate", 1]]',
    true
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robot with wired power are less likely to have heads with camera input

In [14387]:
%%sql

-- Test: one level down
CALL add_discrete_effect(
    'Wired-powered robots are less likely to have camera-input heads',
    'Robot',
    '[["Power Source", "Wired"]]',
    '[["Head", "Head"], ["Input"]]',	
    '[["Camera", -3]]',
    true
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with camera input heads are more likely to have wider necks

In [14388]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with camera input heads are more likely to have wider necks',
    'Robot',
    '[["Head", "Head"], ["Input", "Camera"]]',
    '[["Neck Width"]]',
    5, 0, 0
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered heads

In [14389]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered heads',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Head", "Head"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered torsos

In [14390]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered torsos',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Torso", "Torso"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered left legs

In [14391]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Left Leg", "Leg"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered right legs

In [14392]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Right Leg", "Leg"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered right hands

In [14393]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Right Arm", "Arm"], ["Hand", "Hand"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered left hands

In [14394]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Left Arm", "Arm"], ["Hand", "Hand"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered right sub-arm hands

In [14395]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Right Arm", "Arm"], ["Sub-Arm", "Yes"], ["Hand", "Hand"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with nuclear cores have high-powered left sub-arm hands

In [14396]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '[["Left Arm", "Arm"], ["Sub-Arm", "Yes"], ["Hand", "Hand"], ["Power-Level"]]',
    90, 6, 4
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with steel torsos have a higher weight

In [14397]:
%%sql

-- Test: continous effect on an attribute one level up
CALL add_continuous_effect(
    'Robots with steel torsos have a higher weight',
    'Robot',
    '[["Torso", "Torso"], ["Material", "Steel"]]',
    '[["Weight"]]',
    50, 0, 2
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Robots with iron torsos have a higher weight

In [14398]:
%%sql

CALL add_continuous_effect(
    'Robots with iron torsos have a higher weight',
    'Robot',
    '[["Torso", "Torso"], ["Material", "Iron"]]',
    '[["Weight"]]',
    100, 0, 3
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Grappler hands are more likely to be on metal arms

In [14399]:
%%sql

-- Test: discrete effect on an attribute one level up
CALL add_discrete_effect(
    'Grappler hands are more likely to be on metal arms',
    'Arm',
    '[["Hand", "Hand"], ["Attachment", "Grappler"]]',
    '[["Material"]]',
    '[["Steel", 5],
      ["Aluminum", 5],
      ["Bronze", 5],
      ["Iron", 5],
      ["Carbon-Fiber", 5]]',
    true
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Heads with kazoo outputs can be painted rainbow

In [14400]:
%%sql

-- Test: discrete effect that adds new spans
CALL add_discrete_effect(
    'Heads with kazoo outputs can be painted rainbow',
    'Head',
    '[["Output", "Kazoo"]]',
    '[["Paint", "Paint"], ["Color"]]',
    '[["Rainbow", 100]]'
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate

In [14401]:
%%sql

SELECT 
  pa.id,
  va.name AS variant_attr_name,
  s.label AS subvariant_span_label,
  parent_va.name AS parent_variant_attr_name
FROM possible_attr pa
JOIN variant_attr va 
  ON pa.variant_attr_id = va.id
LEFT JOIN subvariant_span ss 
  ON pa.subvariant_span_id = ss.id
LEFT JOIN span s 
  ON ss.span_id = s.id
LEFT JOIN possible_attr parent_pa 
  ON pa.parent_possible_attr_id = parent_pa.id
LEFT JOIN variant_attr parent_va 
  ON parent_pa.variant_attr_id = parent_va.id
ORDER BY pa.id;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

43 rows affected.

id,variant_attr_name,subvariant_span_label,parent_variant_attr_name
1,Gender,None,None
2,Hair Color,None,None
3,Height,None,None
4,Attachment,None,None
5,Material,None,None
6,Power-Level,None,None
7,Shape,None,None
8,Paint,None,None
9,Color,Paint,Paint
10,Material,None,None


### Get Variant

In [14402]:
%%sql

SELECT * FROM get_variant('Robot', 6);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

200 rows affected.

depth,address,activates,effected_by
1,"[""Power Source""]","['All diesel-powered robots have a roller skate on their left foot', 'Robots with nuclear cores have high-powered heads', 'Robots with nuclear cores have high-powered left hands', 'Robots with nuclear cores have high-powered left legs', 'Robots with nuclear cores have high-powered left sub-arm hands', 'Robots with nuclear cores have high-powered right hands', 'Robots with nuclear cores have high-powered right legs', 'Robots with nuclear cores have high-powered right sub-arm hands', 'Robots with nuclear cores have high-powered torsos', 'Wired-powered robots are less likely to have camera-input heads']",None
1,"[""Head""]",None,None
2,"[[""Head"", ""Head""], ""Input""]",['Robots with camera input heads are more likely to have wider necks'],['Wired-powered robots are less likely to have camera-input heads']
2,"[[""Head"", ""Head""], ""Output""]",None,None
2,"[[""Head"", ""Head""], ""Material""]",None,None
2,"[[""Head"", ""Head""], ""Power-Level""]",None,['Robots with nuclear cores have high-powered heads']
2,"[[""Head"", ""Head""], ""Paint""]",None,None
3,"[[""Head"", ""Head""], [""Paint"", ""Paint""], ""Color""]",None,None
4,"[[""Head"", ""Head""], [""Paint"", ""Paint""], [""Color"", ""Custom""], ""R""]",None,None
4,"[[""Head"", ""Head""], [""Paint"", ""Paint""], [""Color"", ""Custom""], ""G""]",None,None


### Generate Robots

In [14403]:
%%sql

CALL generate_entity_group('Robot', 'Test Robots 1', 'Robot %s', 10);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Inspect Output

In [14404]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Robots 1', 0);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

750 rows affected.

entity_name,entity_state_id,address,numeric_value,span,activates,effected_by,locked
Robot 1,11,"[[""Power Source""]]",None,Wired,"[""Wired-powered robots are less likely to have camera-input heads""]",[],False
Robot 1,11,"[[""Head""]]",None,Head,[],[],False
Robot 1,11,"[[""Head""], [""Input""]]",None,None,[],[],False
Robot 1,11,"[[""Head""], [""Output""]]",None,X-Ray,[],[],False
Robot 1,11,"[[""Head""], [""Material""]]",None,PVC,[],[],False
Robot 1,11,"[[""Head""], [""Power-Level""]]",51.91,None,[],[],False
Robot 1,11,"[[""Head""], [""Paint""]]",None,Paint,[],[],False
Robot 1,11,"[[""Head""], [""Paint""], [""Color""]]",None,Orange,[],[],False
Robot 1,11,"[[""Head""], [""Paint""], [""Sheen""]]",None,Glossy,[],[],False
Robot 1,11,"[[""Neck Width""]]",1.1,None,[],[],False
